# README del Notebook: Balance Virtual y Pronóstico de Entrada por Válvula
Este notebook construye un Balance Virtual de Gas en la red secundaria (válvulas de anillo), pronosticando el Volumen Corregido de Entrada en los periodos sin macromedición y recalculando pérdidas e índices asociados. Además, genera resúmenes comparativos y visualizaciones por válvula.

## Objetivos
- Integrar fuentes de datos históricas (balances, macromedición, usuarios).
- Construir un dataset maestro con variables de entrada/salida y contexto.
- Entrenar modelos por válvula y pronosticar `VOLUMEN_ENTRADA_FINAL` tras la fecha de retiro del macromedidor.
- Recalcular `PERDIDAS_FINAL` e `INDICE_PERDIDAS_FINAL` en el horizonte de pronóstico.
- Generar resúmenes y comparativos por válvula; crear gráficas de seguimiento.

## Datos de entrada requeridos (csv en la carpeta del notebook)
- `Balances.csv`: histórico de balance por válvula (columnas esperadas incluyen AÑO, MES o nombre de mes, ENTRADA_VOLUMEN_MEDIDO_MES, SALIDA_CONSUMO_FACTURADO_MES, DIFERENCIA_PERDIDAS, INDICE_PERDIDAS, PRESION_PROMEDIO_MES, TEMPERATURA_PROMEDIO_MES, FACTOR_CORRECCION_PROMEDIO_MES, CODIGO VALVULA REFERENCIA).
- `Variables_Macromedición_Teleges.csv`: macromedición minuto a minuto (campos `ESTAMPA_TIEMPO`, `CODIGO VALVULA REFERENCIA`, `PRESION`, `TEMPERATURA`, `KPT`, `VOLUMEN_CORREGIDO`).
- `Variables_Usuarios.csv`: consumos por usuario y periodo (campos `ID_USUARIO`, `PERIODO`, `CONSUMO`, `PRESION_SISTEMA`, `KPT_SISTEMA`, `GRUPO_USUARIO`, `ESTRATO`, `CLASE_SERVICIO`, `CODIGO VALVULA REFERENCIA`).
- `Datos_Entrada.csv`: fechas de retiro por válvula y cantidad de periodos a pronosticar (campos `CODIGO VALVULA REFERENCIA`, `FECHA RETIRO/TRASLADO`, `CANTIDAD_PERIODOS_PRONOSTICO`).

## Flujo del notebook (celdas principales)
1) Carga e instalación de librerías (Celdas 4–6).
2) Carga de datasets iniciales (Celda 8).
3) Agregación de macromedición a mensual (Celda 10):
   - Genera `Macromedicion_Mensual.csv` y `Macromedicion_Mensual_Simple.csv`.
   - Estadísticas por válvula y periodo.
4) Agregación de usuarios por válvula/mes (Celda 11):
   - Genera `Usuarios_Por_Valvula.csv` y `Usuarios_Por_Valvula_Simple.csv`.
   - Resumen estadístico por válvula.
5) Construcción del Dataset Maestro (Celda 12):
   - Unifica macromedición mensual y usuarios; integra balance histórico.
   - Crea variables consolidadas: `VOLUMEN_ENTRADA_FINAL`, `VOLUMEN_SALIDA_FINAL`, `PERDIDAS_FINAL`, `INDICE_PERDIDAS_FINAL`, `PRESION_FINAL`, `TEMPERATURA_FINAL`, `KPT_FINAL`.
   - Marca `TIENE_MACROMEDIDOR`, `PERIODO_A_PREDECIR`, `MESES_DESDE_RETIRO`.
   - Genera: `Dataset_Maestro_Balances.csv`, `Dataset_Train.csv` (periodos con macromedidor), `Dataset_Prediccion.csv` (periodos a predecir), `Resumen_Valvulas.csv`.
6) Verificación del dataset de entrenamiento (Celda 13).
7) Entrenamiento y Pronóstico (Celda 15):
   - Modelos por válvula: `Prophet` (serie de tiempo) y `LightGBM` (features).
   - Umbrales flexibles: se entrena Prophet si hay ≥6 puntos; LightGBM si hay ≥6 puntos y features suficientes.
   - Fallback ingenuo (media de últimos 3 valores) cuando la historia es corta o hay fallos.
   - Guardados: `Pronosticos.csv` con `PRED_ENTRADA`, `PRED_SALIDA`, `PRED_PERDIDAS`, `PRED_INDICE_PERDIDAS`; `Metrics.csv` con métricas de LightGBM por válvula si hubo test temporal.
8) Fusión de pronósticos con el dataset maestro (Celda 16):
   - Reemplaza `VOLUMEN_ENTRADA_FINAL` y `VOLUMEN_SALIDA_FINAL` en periodos a predecir.
   - Recalcula `PERDIDAS_FINAL` e `INDICE_PERDIDAS_FINAL`.
   - Guarda `Predicciones_Con_Balance.csv`.
9) Verificación integral de datos (Celda 18): historia, periodos a predecir, retiro vs último periodo, válvulas con poca historia.
10) Resumen por válvula y gráficas (Celdas 19–21):
    - `Resumen_Pronostico_Valvulas.csv` con totales y promedios del horizonte de pronóstico.
    - Gráficas por válvula (HTML y PNG) en `graficas_valvulas/`: Entrada vs Salida; Índice de Pérdidas.
11) Tabla comparativa (Celda 22):
    - `Comparativo_Valvulas.csv` con KPIs, rankings y métricas (si disponibles).
    - Genera `Resumen_Pronostico_Valvulas.csv` automáticamente si falta (a partir de `Predicciones_Con_Balance.csv`).
12) Dashboard consolidado (Celdas 23–24):
    - `graficas_valvulas/dashboard.html` con tabla comparativa y iframes de gráficas.
13) Limpieza opcional (Celdas 25–26):
    - Bandera `MODO_LIMPIEZA = True` para borrar derivados y gráficas tras la validación.

## Archivos generados (salidas)
- `Macromedicion_Mensual.csv`, `Macromedicion_Mensual_Simple.csv`: agregaciones de macromedición.
- `Usuarios_Por_Valvula.csv`, `Usuarios_Por_Valvula_Simple.csv`: consumos y promedios por válvula.
- `Dataset_Maestro_Balances.csv`: dataset final consolidado.
- `Dataset_Train.csv`: periodos con macromedidor (para entrenamiento).
- `Dataset_Prediccion.csv`: periodos a predecir (tras retiro).
- `Pronosticos.csv`: pronósticos por válvula y periodo (entrada/salida/pérdidas/índice).
- `Metrics.csv`: métricas LightGBM por válvula (si hubo test).
- `Predicciones_Con_Balance.csv`: dataset maestro con valores pronosticados.
- `Resumen_Pronostico_Valvulas.csv`: resumen KPIs del horizonte de pronóstico por válvula.
- `Comparativo_Valvulas.csv`: tabla comparativa con rankings e indicadores.
- `graficas_valvulas/*`: HTML y PNG de gráficas por válvula, y `dashboard.html`.

## Parámetros y banderas
- `MODO_LIMPIEZA` (Celda 26): True para eliminar derivados y gráficas; False para conservar salidas.
- Umbrales de entrenamiento: Prophet y LightGBM requieren ≥6 puntos; fallback si no se cumplen.

## Cómo ejecutar end-to-end
1) Asegura que los cuatro datasets de entrada estén presentes (`Balances.csv`, `Variables_Macromedición_Teleges.csv`, `Variables_Usuarios.csv`, `Datos_Entrada.csv`).
2) Ejecuta secuencialmente todas las celdas (de 4 a 24) para crear agregados, maestro, entrenar y fusionar.
3) Revisa las salidas clave: `Predicciones_Con_Balance.csv`, `Pronosticos.csv`, `Comparativo_Valvulas.csv`, `graficas_valvulas/dashboard.html`.
4) Opcional: ejecuta la Celda 26 con `MODO_LIMPIEZA = True` para limpiar derivados tras validar.

## Solución de problemas (FAQ)
- FileNotFoundError de `Resumen_Pronostico_Valvulas.csv`: la Celda 22 lo genera automáticamente a partir de `Predicciones_Con_Balance.csv`. Asegura haber ejecutado la Celda 16 primero.
- Errores de resta con strings al fusionar: la Celda 16 normaliza tipos numéricos (`decimal=','` y `to_numeric`).
- Series con muy pocos puntos: se usa fallback ingenuo; `VALVULA_5` es un ejemplo con historia <6.
- Prophet con pocas observaciones: se reduce `n_changepoints` automáticamente; si falla, se usa fallback.
- Diferencias de formato decimal: todas las lecturas usan `sep=';'` y `decimal=','` para consistencia.

## Notas y criterios
- No se modifican los archivos iniciales; los derivados se regeneran en cada corrida.
- Las métricas de LightGBM sólo se calculan si existe test temporal (≥2 puntos).
- Los gráficos y dashboard son opcionales; se pueden limpiar con `MODO_LIMPIEZA`.
- El comparativo ordena por menor `% de pérdidas sobre entrada` y `INDICE_PERDIDAS_MEAN`.


# Karly Velasquez Acosta - 3023368928 - velasquezacostakarly@gmail.com
# Julian Santiago Pico Pinzon - 3043089479 - julpic08@gmail.com


## Descripción de los datos

El objetivo principal de este proyecto es desarrollar un Modelo Predictivo de Series de Tiempo que pueda estimar el Volumen Corregido (Entrada) de gas en puntos de la red secundaria (válvulas de anillo) donde la medición física en tiempo real ha cesado.

Este pronóstico de volumen de entrada permitirá calcular un Balance Virtual de Gas al compararlo con el consumo facturado de los usuarios (Volumen Salida), identificando y cuantificando las pérdidas o desbalances en el sistema

## proposito

- Pronosticar el Volumen Corregido para cada válvula de anillo en el horizonte de pronóstico (a partir de la fecha de retiro del macromedidor).

- Comparar y Evaluar el desempeño de tres niveles de modelos: SARIMA (Clásico), LightGBM (ML Potente), y TFT (Deep Learning Avanzado).

- Realizar el Balance Virtual y determinar el Índice de Pérdidas (%) proyectado para el periodo sin medición.

#Cargar librerias e instalar librerias


In [1]:
#!pip install lightgbm
#!pip install prophet

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
from xgboost import XGBRegressor
from prophet import Prophet

C:\Users\karly\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

#Cargar conjunto de datos

In [4]:
try:
    df_balances = pd.read_csv('Balances.csv', sep=';', encoding='latin-1')
    df_datos_entrada = pd.read_csv('Datos_Entrada.csv', sep=';', encoding='latin-1')
    df_macromedicion = pd.read_csv('Variables_Macromedición_Teleges.csv', sep=';', encoding='latin-1')
    df_usuarios = pd.read_csv('Variables_Usuarios.csv', sep=';', encoding='latin-1')

    print("Carga de datos exitosa")
except Exception as e:
    print(f"Error al cargar los archivos: {e}")

Carga de datos exitosa


#Analizar y visualizar datos

In [5]:
import pandas as pd
import numpy as np

print("=" * 80)
print("AGREGACIÓN MACROMEDICIÓN: MINUTO A MINUTO → MENSUAL")
print("=" * 80)

# ============================================================================
# CARGAR Y PREPARAR DATOS
# ============================================================================
print("\n1. Cargando datos...")
df = pd.read_csv('Variables_Macromedición_Teleges.csv', sep=';', encoding='latin-1')
print(f"   ✓ Datos cargados: {df.shape}")
print(f"   ✓ Columnas: {list(df.columns)}")

# Limpiar nombres de columnas
df.columns = df.columns.str.strip()

# ============================================================================
# CONVERTIR FECHA Y COLUMNAS NUMÉRICAS
# ============================================================================
print("\n2. Convirtiendo tipos de datos...")

# Convertir fecha
df['ESTAMPA_TIEMPO'] = pd.to_datetime(df['ESTAMPA_TIEMPO'],
                                      format='%d/%m/%Y %H:%M',
                                      errors='coerce')

# Función para convertir coma decimal a punto
def to_numeric(value):
    if pd.isna(value):
        return np.nan
    if isinstance(value, (int, float)):
        return float(value)
    try:
        return float(str(value).replace(',', '.'))
    except:
        return np.nan

# Convertir columnas numéricas
numeric_cols = ['PRESION', 'TEMPERATURA', 'KPT', 'VOLUMEN_CORREGIDO']
for col in numeric_cols:
    df[col] = df[col].apply(to_numeric)
    print(f"   ✓ {col} convertido a numérico")

# Crear columnas de periodo
df['AÑO_MES'] = df['ESTAMPA_TIEMPO'].dt.to_period('M')
df['AÑO'] = df['ESTAMPA_TIEMPO'].dt.year
df['MES'] = df['ESTAMPA_TIEMPO'].dt.month

print(f"\n   Rango de fechas: {df['ESTAMPA_TIEMPO'].min()} → {df['ESTAMPA_TIEMPO'].max()}")
print(f"   Válvulas únicas: {df['CODIGO VALVULA REFERENCIA'].nunique()}")

# ============================================================================
# AGREGACIÓN MENSUAL
# ============================================================================
print("\n3. Agregando datos por mes...")

# Definir agregaciones
agregaciones = {
    'PRESION': ['mean', 'std', 'min', 'max'],
    'TEMPERATURA': ['mean', 'std', 'min', 'max'],
    'KPT': ['mean', 'std', 'min', 'max'],
    'VOLUMEN_CORREGIDO': ['sum', 'mean', 'std', 'min', 'max'],  # SUMA es el total del mes
    'ESTAMPA_TIEMPO': 'count'  # Cantidad de registros
}

# Agrupar por válvula y mes
df_mensual = df.groupby(['CODIGO VALVULA REFERENCIA', 'AÑO_MES']).agg(agregaciones).reset_index()

# Aplanar nombres de columnas
df_mensual.columns = ['_'.join(col).strip('_') if col[1] else col[0]
                      for col in df_mensual.columns.values]

# Renombrar columnas para mayor claridad
df_mensual.rename(columns={
    'CODIGO VALVULA REFERENCIA': 'VALVULA',
    'ESTAMPA_TIEMPO_count': 'NUM_REGISTROS',
    'PRESION_mean': 'PRESION_PROMEDIO',
    'PRESION_std': 'PRESION_DESVIACION',
    'PRESION_min': 'PRESION_MIN',
    'PRESION_max': 'PRESION_MAX',
    'TEMPERATURA_mean': 'TEMPERATURA_PROMEDIO',
    'TEMPERATURA_std': 'TEMPERATURA_DESVIACION',
    'TEMPERATURA_min': 'TEMPERATURA_MIN',
    'TEMPERATURA_max': 'TEMPERATURA_MAX',
    'KPT_mean': 'KPT_PROMEDIO',
    'KPT_std': 'KPT_DESVIACION',
    'KPT_min': 'KPT_MIN',
    'KPT_max': 'KPT_MAX',
    'VOLUMEN_CORREGIDO_sum': 'VOLUMEN_TOTAL_MES',
    'VOLUMEN_CORREGIDO_mean': 'VOLUMEN_PROMEDIO',
    'VOLUMEN_CORREGIDO_std': 'VOLUMEN_DESVIACION',
    'VOLUMEN_CORREGIDO_min': 'VOLUMEN_MIN',
    'VOLUMEN_CORREGIDO_max': 'VOLUMEN_MAX'
}, inplace=True)

# Convertir periodo a formato legible
df_mensual['PERIODO'] = df_mensual['AÑO_MES'].astype(str)
df_mensual['AÑO'] = df_mensual['AÑO_MES'].dt.year
df_mensual['MES'] = df_mensual['AÑO_MES'].dt.month

# Reordenar columnas
columnas_orden = ['VALVULA', 'PERIODO', 'AÑO', 'MES'] + \
                 [col for col in df_mensual.columns if col not in ['VALVULA', 'PERIODO', 'AÑO', 'MES', 'AÑO_MES']]
df_mensual = df_mensual[columnas_orden]

print(f"   ✓ Agregación completada: {df_mensual.shape}")

# ============================================================================
# ESTADÍSTICAS Y VALIDACIÓN
# ============================================================================
print("\n4. Resumen de datos agregados:")
print("\n   Registros por válvula:")
print(df_mensual.groupby('VALVULA').size())

print("\n   Rango temporal por válvula:")
rango = df_mensual.groupby('VALVULA')['PERIODO'].agg(['min', 'max', 'count'])
rango.columns = ['Primer_Mes', 'Último_Mes', 'Total_Meses']
print(rango)

print("\n   Estadísticas de volumen total por mes:")
print(df_mensual['VOLUMEN_TOTAL_MES'].describe())

print("\n   Muestra de datos (primeros 10 registros):")
print(df_mensual[['VALVULA', 'PERIODO', 'PRESION_PROMEDIO', 'TEMPERATURA_PROMEDIO',
                  'KPT_PROMEDIO', 'VOLUMEN_TOTAL_MES', 'NUM_REGISTROS']].head(10))

# ============================================================================
# GUARDAR RESULTADOS
# ============================================================================
print("\n5. Guardando resultados...")

# Guardar archivo principal
df_mensual.to_csv('Macromedicion_Mensual.csv', index=False, sep=';', decimal=',', encoding='latin-1')
print(f"   ✓ Archivo guardado: Macromedicion_Mensual.csv")

# Guardar versión simplificada (solo promedios)
df_simple = df_mensual[['VALVULA', 'PERIODO', 'AÑO', 'MES',
                        'PRESION_PROMEDIO', 'TEMPERATURA_PROMEDIO',
                        'KPT_PROMEDIO', 'VOLUMEN_TOTAL_MES', 'NUM_REGISTROS']]
df_simple.to_csv('Macromedicion_Mensual_Simple.csv', index=False, sep=';', decimal=',', encoding='latin-1')
print(f"   ✓ Archivo simplificado: Macromedicion_Mensual_Simple.csv")

print("\n" + "=" * 80)
print("PROCESO COMPLETADO ✓")
print("=" * 80)
print(f"\nDimensiones finales: {df_mensual.shape}")
print(f"Columnas creadas: {list(df_mensual.columns)}")
print(f"\nArchivos generados:")
print("  • Macromedicion_Mensual.csv (completo con estadísticas)")
print("  • Macromedicion_Mensual_Simple.csv (solo promedios)")

AGREGACIÓN MACROMEDICIÓN: MINUTO A MINUTO → MENSUAL

1. Cargando datos...
   ✓ Datos cargados: (565502, 6)
   ✓ Columnas: ['CODIGO VALVULA REFERENCIA', 'ESTAMPA_TIEMPO', 'PRESION', 'TEMPERATURA', 'KPT', 'VOLUMEN_CORREGIDO']

2. Convirtiendo tipos de datos...
   ✓ PRESION convertido a numérico
   ✓ TEMPERATURA convertido a numérico
   ✓ KPT convertido a numérico
   ✓ VOLUMEN_CORREGIDO convertido a numérico

   Rango de fechas: 2024-05-18 00:00:00 → 2025-07-16 00:00:00
   Válvulas únicas: 5

3. Agregando datos por mes...
   ✓ Agregación completada: (32, 22)

4. Resumen de datos agregados:

   Registros por válvula:
VALVULA
VALVULA_1    7
VALVULA_2    8
VALVULA_3    7
VALVULA_4    6
VALVULA_5    4
dtype: int64

   Rango temporal por válvula:
          Primer_Mes Último_Mes  Total_Meses
VALVULA                                     
VALVULA_1    2024-07    2025-01            7
VALVULA_2    2024-05    2024-12            8
VALVULA_3    2025-01    2025-07            7
VALVULA_4    2024-06    20

In [6]:
import pandas as pd
import numpy as np

print("=" * 80)
print("AGREGACIÓN USUARIOS: POR USUARIO → POR VÁLVULA Y MES")
print("=" * 80)

# ============================================================================
# CARGAR DATOS
# ============================================================================
print("\n1. Cargando datos...")
df = pd.read_csv('Variables_Usuarios.csv', sep=';', encoding='latin-1')
print(f"   ✓ Datos cargados: {df.shape}")
print(f"   ✓ Columnas: {list(df.columns)}")

# Limpiar nombres de columnas
df.columns = df.columns.str.strip()

print(f"\n   Total registros: {len(df):,}")
print(f"   Válvulas únicas: {df['CODIGO VALVULA REFERENCIA'].nunique()}")
print(f"   Usuarios únicos: {df['ID_USUARIO'].nunique()}")
print(f"   Periodos: {df['PERIODO'].min()} → {df['PERIODO'].max()}")

# ============================================================================
# CONVERTIR COLUMNAS NUMÉRICAS
# ============================================================================
print("\n2. Convirtiendo columnas numéricas...")

# Función para convertir coma decimal a punto
def to_numeric(value):
    if pd.isna(value):
        return np.nan
    if isinstance(value, (int, float)):
        return float(value)
    try:
        return float(str(value).replace(',', '.'))
    except:
        return np.nan

# Convertir columnas numéricas
numeric_cols = ['PRESION_SISTEMA', 'KPT_SISTEMA', 'CONSUMO']
for col in numeric_cols:
    if col in df.columns:
        df[col] = df[col].apply(to_numeric)
        print(f"   ✓ {col} convertido a numérico")

# ============================================================================
# AGREGAR POR VÁLVULA Y MES
# ============================================================================
print("\n3. Agregando por válvula y mes...")

# Definir agregaciones
agregaciones = {
    'CONSUMO': ['sum', 'mean', 'std', 'min', 'max'],  # SUMA es el total de consumo de la válvula
    'PRESION_SISTEMA': ['mean', 'std', 'min', 'max'],
    'KPT_SISTEMA': ['mean', 'std', 'min', 'max'],
    'ID_USUARIO': 'count',  # Cantidad de usuarios
    'GRUPO_USUARIO': lambda x: x.mode()[0] if len(x.mode()) > 0 else x.iloc[0],
    'ESTRATO': lambda x: x.mode()[0] if len(x.mode()) > 0 else x.iloc[0],
    'CLASE_SERVICIO': lambda x: x.mode()[0] if len(x.mode()) > 0 else x.iloc[0]
}

# Agrupar
df_valvula = df.groupby(['CODIGO VALVULA REFERENCIA', 'PERIODO']).agg(agregaciones).reset_index()

# Aplanar nombres de columnas
df_valvula.columns = ['_'.join(col).strip('_') if col[1] else col[0]
                      for col in df_valvula.columns.values]

# Renombrar columnas para mayor claridad
df_valvula.rename(columns={
    'CODIGO VALVULA REFERENCIA': 'VALVULA',
    'CONSUMO_sum': 'CONSUMO_TOTAL_VALVULA',  # Este es el volumen de salida
    'CONSUMO_mean': 'CONSUMO_PROMEDIO_USUARIO',
    'CONSUMO_std': 'CONSUMO_DESVIACION_USUARIO',
    'CONSUMO_min': 'CONSUMO_MIN_USUARIO',
    'CONSUMO_max': 'CONSUMO_MAX_USUARIO',
    'PRESION_SISTEMA_mean': 'PRESION_PROMEDIO',
    'PRESION_SISTEMA_std': 'PRESION_DESVIACION',
    'PRESION_SISTEMA_min': 'PRESION_MIN',
    'PRESION_SISTEMA_max': 'PRESION_MAX',
    'KPT_SISTEMA_mean': 'KPT_PROMEDIO',
    'KPT_SISTEMA_std': 'KPT_DESVIACION',
    'KPT_SISTEMA_min': 'KPT_MIN',
    'KPT_SISTEMA_max': 'KPT_MAX',
    'ID_USUARIO_count': 'NUM_USUARIOS',
    'GRUPO_USUARIO_<lambda>': 'GRUPO_MODAL',
    'ESTRATO_<lambda>': 'ESTRATO_MODAL',
    'CLASE_SERVICIO_<lambda>': 'CLASE_SERVICIO_MODAL'
}, inplace=True)

# Crear columnas de año y mes
df_valvula['PERIODO'] = df_valvula['PERIODO'].astype(str)
df_valvula['AÑO'] = df_valvula['PERIODO'].str[:4].astype(int)
df_valvula['MES'] = df_valvula['PERIODO'].str[4:6].astype(int)

# Reordenar columnas
columnas_orden = ['VALVULA', 'PERIODO', 'AÑO', 'MES',
                 'CONSUMO_TOTAL_VALVULA', 'NUM_USUARIOS'] + \
                 [col for col in df_valvula.columns
                  if col not in ['VALVULA', 'PERIODO', 'AÑO', 'MES',
                                'CONSUMO_TOTAL_VALVULA', 'NUM_USUARIOS']]
df_valvula = df_valvula[columnas_orden]

print(f"   ✓ Agregación completada: {df_valvula.shape}")

# ============================================================================
# ESTADÍSTICAS Y VALIDACIÓN
# ============================================================================
print("\n4. Resumen de datos agregados:")

print("\n   Registros por válvula:")
print(df_valvula.groupby('VALVULA').size())

print("\n   Rango temporal por válvula:")
rango = df_valvula.groupby('VALVULA')['PERIODO'].agg(['min', 'max', 'count'])
rango.columns = ['Primer_Periodo', 'Último_Periodo', 'Total_Meses']
print(rango)

print("\n   Estadísticas de consumo total por válvula:")
print(df_valvula.groupby('VALVULA')['CONSUMO_TOTAL_VALVULA'].describe())

print("\n   Muestra de datos (primeros 10 registros):")
print(df_valvula[['VALVULA', 'PERIODO', 'CONSUMO_TOTAL_VALVULA', 'NUM_USUARIOS',
                  'PRESION_PROMEDIO', 'KPT_PROMEDIO']].head(10))

# ============================================================================
# GUARDAR RESULTADOS
# ============================================================================
print("\n5. Guardando resultados...")

# Guardar archivo completo
df_valvula.to_csv('Usuarios_Por_Valvula.csv', index=False, sep=';', decimal=',', encoding='latin-1')
print(f"   ✓ Archivo completo: Usuarios_Por_Valvula.csv")

# Guardar versión simplificada (solo promedios y totales)
df_simple = df_valvula[['VALVULA', 'PERIODO', 'AÑO', 'MES',
                        'CONSUMO_TOTAL_VALVULA', 'NUM_USUARIOS',
                        'PRESION_PROMEDIO', 'KPT_PROMEDIO',
                        'GRUPO_MODAL', 'CLASE_SERVICIO_MODAL']]
df_simple.to_csv('Usuarios_Por_Valvula_Simple.csv', index=False, sep=';', decimal=',', encoding='latin-1')
print(f"   ✓ Archivo simplificado: Usuarios_Por_Valvula_Simple.csv")

# Guardar resumen estadístico
resumen = df_valvula.groupby('VALVULA').agg({
    'CONSUMO_TOTAL_VALVULA': ['sum', 'mean'],
    'NUM_USUARIOS': 'mean',
    'PRESION_PROMEDIO': 'mean',
    'KPT_PROMEDIO': 'mean',
    'PERIODO': 'count'
}).reset_index()
resumen.columns = ['_'.join(col).strip('_') for col in resumen.columns.values]
resumen.to_csv('Resumen_Por_Valvula.csv', index=False, sep=';', decimal=',', encoding='latin-1')
print(f"   ✓ Resumen estadístico: Resumen_Por_Valvula.csv")

print("\n" + "=" * 80)
print("PROCESO COMPLETADO ✓")
print("=" * 80)
print(f"\nDimensiones finales: {df_valvula.shape}")
print(f"Columnas creadas: {list(df_valvula.columns)}")
print(f"\nArchivos generados:")
print("  • Usuarios_Por_Valvula.csv (completo con estadísticas)")
print("  • Usuarios_Por_Valvula_Simple.csv (solo promedios y totales)")
print("  • Resumen_Por_Valvula.csv (resumen por válvula)")
print(f"\n💡 Nota: CONSUMO_TOTAL_VALVULA es la suma de consumos de todos los usuarios")
print("         (equivale al volumen de salida en el balance)")

AGREGACIÓN USUARIOS: POR USUARIO → POR VÁLVULA Y MES

1. Cargando datos...
   ✓ Datos cargados: (6768, 10)
   ✓ Columnas: ['CODIGO VALVULA REFERENCIA', 'ID_USUARIO', 'GRUPO_USUARIO', 'ESTRATO', 'CLASE_SERVICIO', 'PRESION_SISTEMA', 'KPT_SISTEMA', 'TIPO_MEDIDOR', 'PERIODO', 'CONSUMO']

   Total registros: 6,768
   Válvulas únicas: 5
   Usuarios únicos: 443
   Periodos: 202407 → 202511

2. Convirtiendo columnas numéricas...
   ✓ PRESION_SISTEMA convertido a numérico
   ✓ KPT_SISTEMA convertido a numérico
   ✓ CONSUMO convertido a numérico

3. Agregando por válvula y mes...
   ✓ Agregación completada: (72, 21)

4. Resumen de datos agregados:

   Registros por válvula:
VALVULA
VALVULA_1    15
VALVULA_2    17
VALVULA_3     9
VALVULA_4    16
VALVULA_5    15
dtype: int64

   Rango temporal por válvula:
          Primer_Periodo Último_Periodo  Total_Meses
VALVULA                                             
VALVULA_1         202409         202511           15
VALVULA_2         202407         20

In [7]:
import pandas as pd
import numpy as np
from datetime import datetime

print("=" * 80)
print("CREACIÓN DE DATASET MAESTRO DE BALANCES")
print("=" * 80)

# ============================================================================
# PASO 1: CARGAR TODOS LOS DATASETS
# ============================================================================
print("\n1. Cargando datasets...")

df_balances = pd.read_csv('Balances.csv', sep=';', encoding='latin-1')
df_datos_entrada = pd.read_csv('Datos_Entrada.csv', sep=';', encoding='latin-1')
df_macro = pd.read_csv('Macromedicion_Mensual_Simple.csv', sep=';', encoding='latin-1')
df_usuarios = pd.read_csv('Usuarios_Por_Valvula_Simple.csv', sep=';', encoding='latin-1')

print(f"   ✓ Balances históricos: {df_balances.shape}")
print(f"   ✓ Datos entrada: {df_datos_entrada.shape}")
print(f"   ✓ Macromedición mensual: {df_macro.shape}")
print(f"   ✓ Usuarios por válvula: {df_usuarios.shape}")

# ============================================================================
# PASO 2: PREPARAR DATOS DE ENTRADA (FECHAS DE RETIRO)
# ============================================================================
print("\n2. Procesando fechas de retiro del macromedidor...")

df_datos_entrada.columns = df_datos_entrada.columns.str.strip()

# Función de conversión numérica
def to_numeric(value):
    if pd.isna(value):
        return np.nan
    if isinstance(value, (int, float)):
        return float(value)
    try:
        return float(str(value).replace(',', '.'))
    except:
        return np.nan

# Convertir fecha de retiro
for formato in ['%d/%m/%Y', '%Y-%m-%d', '%d-%m-%Y']:
    try:
        df_datos_entrada['FECHA_RETIRO'] = pd.to_datetime(
            df_datos_entrada['FECHA RETIRO/TRASLADO'],
            format=formato,
            errors='coerce'
        )
        if df_datos_entrada['FECHA_RETIRO'].notna().sum() > 0:
            break
    except:
        continue

df_datos_entrada['VALVULA'] = df_datos_entrada['CODIGO VALVULA REFERENCIA']
df_datos_entrada['PERIODO_RETIRO'] = df_datos_entrada['FECHA_RETIRO'].dt.to_period('M').astype(str).str.replace('-', '')
df_datos_entrada['CANTIDAD_PERIODOS_PRONOSTICO'] = df_datos_entrada['CANTIDAD_PERIODOS_PRONOSTICO'].apply(to_numeric)

print("\n   Información de retiro por válvula:")
print(df_datos_entrada[['VALVULA', 'FECHA_RETIRO', 'PERIODO_RETIRO', 'CANTIDAD_PERIODOS_PRONOSTICO']])

# ============================================================================
# PASO 3: PREPARAR BALANCES HISTÓRICOS
# ============================================================================
print("\n3. Preparando balances históricos...")

df_balances.columns = df_balances.columns.str.strip()

# Convertir columnas numéricas
numeric_cols_bal = ['ENTRADA_VOLUMEN_MEDIDO_MES', 'SALIDA_CONSUMO_FACTURADO_MES',
                    'DIFERENCIA_PERDIDAS', 'INDICE_PERDIDAS',
                    'PRESION_PROMEDIO_MES', 'TEMPERATURA_PROMEDIO_MES',
                    'FACTOR_CORRECCION_PROMEDIO_MES']

for col in numeric_cols_bal:
    if col in df_balances.columns:
        df_balances[col] = df_balances[col].apply(to_numeric)

# Crear PERIODO desde AÑO y MES
df_balances['VALVULA'] = df_balances['CODIGO VALVULA REFERENCIA']

# Mapeo de meses en español a números
meses_map = {
    'enero': '01', 'febrero': '02', 'marzo': '03', 'abril': '04',
    'mayo': '05', 'junio': '06', 'julio': '07', 'agosto': '08',
    'septiembre': '09', 'octubre': '10', 'noviembre': '11', 'diciembre': '12'
}

# Si MES está en texto, convertir a número
if df_balances['MES'].dtype == 'object':
    df_balances['MES_NUM'] = df_balances['MES'].str.lower().str.strip().map(meses_map)
else:
    df_balances['MES_NUM'] = df_balances['MES'].apply(lambda x: f'{int(x):02d}' if pd.notna(x) else None)

# Crear PERIODO en formato YYYYMM
df_balances['PERIODO'] = df_balances['AÑO'].astype(str) + df_balances['MES_NUM'].astype(str)

print(f"   Ejemplo de periodos creados: {df_balances['PERIODO'].head().tolist()}")

# Seleccionar columnas relevantes
df_balances_clean = df_balances[[
    'VALVULA', 'PERIODO',
    'ENTRADA_VOLUMEN_MEDIDO_MES', 'SALIDA_CONSUMO_FACTURADO_MES',
    'DIFERENCIA_PERDIDAS', 'INDICE_PERDIDAS',
    'PRESION_PROMEDIO_MES', 'TEMPERATURA_PROMEDIO_MES',
    'FACTOR_CORRECCION_PROMEDIO_MES'
]].copy()

df_balances_clean.rename(columns={
    'ENTRADA_VOLUMEN_MEDIDO_MES': 'VOLUMEN_ENTRADA',
    'SALIDA_CONSUMO_FACTURADO_MES': 'VOLUMEN_SALIDA',
    'DIFERENCIA_PERDIDAS': 'PERDIDAS',
    'INDICE_PERDIDAS': 'INDICE_PERDIDAS',
    'PRESION_PROMEDIO_MES': 'PRESION_BALANCE',
    'TEMPERATURA_PROMEDIO_MES': 'TEMPERATURA_BALANCE',
    'FACTOR_CORRECCION_PROMEDIO_MES': 'KPT_BALANCE'
}, inplace=True)

df_balances_clean['ORIGEN'] = 'BALANCE_HISTORICO'

print(f"   ✓ Balances históricos procesados: {df_balances_clean.shape}")

# ============================================================================
# PASO 4: PREPARAR MACROMEDICIÓN MENSUAL
# ============================================================================
print("\n4. Preparando macromedición mensual...")

df_macro.columns = df_macro.columns.str.strip()

# Convertir numéricas
for col in ['PRESION_PROMEDIO', 'TEMPERATURA_PROMEDIO', 'KPT_PROMEDIO', 'VOLUMEN_TOTAL_MES']:
    if col in df_macro.columns:
        df_macro[col] = df_macro[col].apply(to_numeric)

# Normalizar PERIODO
df_macro['PERIODO'] = df_macro['PERIODO'].astype(str).str.replace('-', '')

df_macro_clean = df_macro[[
    'VALVULA', 'PERIODO',
    'VOLUMEN_TOTAL_MES', 'PRESION_PROMEDIO',
    'TEMPERATURA_PROMEDIO', 'KPT_PROMEDIO', 'NUM_REGISTROS'
]].copy()

df_macro_clean.rename(columns={
    'VOLUMEN_TOTAL_MES': 'VOLUMEN_ENTRADA_MACRO',
    'PRESION_PROMEDIO': 'PRESION_MACRO',
    'TEMPERATURA_PROMEDIO': 'TEMPERATURA_MACRO',
    'KPT_PROMEDIO': 'KPT_MACRO'
}, inplace=True)

print(f"   ✓ Macromedición mensual procesada: {df_macro_clean.shape}")

# ============================================================================
# PASO 5: PREPARAR USUARIOS
# ============================================================================
print("\n5. Preparando usuarios por válvula...")

df_usuarios.columns = df_usuarios.columns.str.strip()

# Convertir numéricas
for col in ['CONSUMO_TOTAL_VALVULA', 'PRESION_PROMEDIO', 'KPT_PROMEDIO']:
    if col in df_usuarios.columns:
        df_usuarios[col] = df_usuarios[col].apply(to_numeric)

df_usuarios['PERIODO'] = df_usuarios['PERIODO'].astype(str)

df_usuarios_clean = df_usuarios[[
    'VALVULA', 'PERIODO',
    'CONSUMO_TOTAL_VALVULA', 'NUM_USUARIOS',
    'PRESION_PROMEDIO', 'KPT_PROMEDIO'
]].copy()

df_usuarios_clean.rename(columns={
    'CONSUMO_TOTAL_VALVULA': 'VOLUMEN_SALIDA_USUARIOS',
    'PRESION_PROMEDIO': 'PRESION_USUARIOS',
    'KPT_PROMEDIO': 'KPT_USUARIOS'
}, inplace=True)

print(f"   ✓ Usuarios procesados: {df_usuarios_clean.shape}")

# ============================================================================
# PASO 6: UNIR TODOS LOS DATOS
# ============================================================================
print("\n6. Uniendo todos los datasets...")

# Unir macromedición con usuarios
df_maestro = df_macro_clean.merge(
    df_usuarios_clean,
    on=['VALVULA', 'PERIODO'],
    how='outer'
)

print(f"   ✓ Unión macro + usuarios: {df_maestro.shape}")

# Unir con balances históricos
df_maestro = df_maestro.merge(
    df_balances_clean,
    on=['VALVULA', 'PERIODO'],
    how='outer',
    suffixes=('', '_HIST')
)

print(f"   ✓ Unión con balances históricos: {df_maestro.shape}")

# ============================================================================
# PASO 7: CREAR VARIABLES FINALES Y CALCULAR BALANCES
# ============================================================================
print("\n7. Creando variables finales...")

# Crear variables consolidadas
df_maestro['VOLUMEN_ENTRADA_FINAL'] = df_maestro['VOLUMEN_ENTRADA'].fillna(
    df_maestro['VOLUMEN_ENTRADA_MACRO']
)

df_maestro['VOLUMEN_SALIDA_FINAL'] = df_maestro['VOLUMEN_SALIDA'].fillna(
    df_maestro['VOLUMEN_SALIDA_USUARIOS']
)

# Calcular balance
df_maestro['PERDIDAS_CALC'] = (df_maestro['VOLUMEN_ENTRADA_FINAL'] -
                                df_maestro['VOLUMEN_SALIDA_FINAL'])

df_maestro['INDICE_PERDIDAS_CALC'] = np.where(
    df_maestro['VOLUMEN_ENTRADA_FINAL'] > 0,
    (df_maestro['PERDIDAS_CALC'] / df_maestro['VOLUMEN_ENTRADA_FINAL']) * 100,
    np.nan
)

# Usar balance histórico si existe, sino el calculado
df_maestro['PERDIDAS_FINAL'] = df_maestro['PERDIDAS'].fillna(df_maestro['PERDIDAS_CALC'])
df_maestro['INDICE_PERDIDAS_FINAL'] = df_maestro['INDICE_PERDIDAS'].fillna(df_maestro['INDICE_PERDIDAS_CALC'])

# Consolidar presión y temperatura
df_maestro['PRESION_FINAL'] = df_maestro['PRESION_BALANCE'].fillna(
    df_maestro['PRESION_MACRO']
).fillna(df_maestro['PRESION_USUARIOS'])

df_maestro['TEMPERATURA_FINAL'] = df_maestro['TEMPERATURA_BALANCE'].fillna(
    df_maestro['TEMPERATURA_MACRO']
)

df_maestro['KPT_FINAL'] = df_maestro['KPT_BALANCE'].fillna(
    df_maestro['KPT_MACRO']
).fillna(df_maestro['KPT_USUARIOS'])

# Filtrar registros sin periodo
print(f"\n   Registros antes de filtrar: {len(df_maestro)}")
df_maestro = df_maestro[df_maestro['PERIODO'].notna()].copy()
df_maestro['PERIODO'] = df_maestro['PERIODO'].astype(str)
print(f"   Registros después de filtrar NaN: {len(df_maestro)}")

# Verificar y limpiar formatos de PERIODO
print(f"\n   Muestras de PERIODO: {df_maestro['PERIODO'].unique()[:10]}")

# Normalizar PERIODO: eliminar guiones y espacios
df_maestro['PERIODO'] = df_maestro['PERIODO'].str.replace('-', '').str.replace(' ', '').str.strip()

# Verificar longitud correcta (debe ser 6 dígitos: YYYYMM)
mask_valido = df_maestro['PERIODO'].str.len() == 6
print(f"   Registros con formato válido (6 dígitos): {mask_valido.sum()}")
print(f"   Registros con formato inválido: {(~mask_valido).sum()}")

if (~mask_valido).sum() > 0:
    print(f"\n   ⚠ PERIODOS inválidos encontrados:")
    print(df_maestro[~mask_valido][['VALVULA', 'PERIODO']].head(10))
    print(f"\n   Eliminando {(~mask_valido).sum()} registros con periodo inválido...")
    df_maestro = df_maestro[mask_valido].copy()

# Crear columnas temporales
df_maestro['AÑO'] = df_maestro['PERIODO'].str[:4].astype(int)
df_maestro['MES'] = df_maestro['PERIODO'].str[4:6].astype(int)
df_maestro['FECHA'] = pd.to_datetime(df_maestro['PERIODO'], format='%Y%m', errors='coerce')

# Ordenar
df_maestro = df_maestro.sort_values(['VALVULA', 'FECHA']).reset_index(drop=True)

# ============================================================================
# PASO 8: MARCAR PERIODOS CON/SIN MACROMEDIDOR
# ============================================================================
print("\n8. Identificando periodos con/sin macromedidor...")

# Unir con fechas de retiro
df_maestro = df_maestro.merge(
    df_datos_entrada[['VALVULA', 'PERIODO_RETIRO', 'CANTIDAD_PERIODOS_PRONOSTICO']],
    on='VALVULA',
    how='left'
)

# Marcar si hay macromedidor
df_maestro['TIENE_MACROMEDIDOR'] = df_maestro['VOLUMEN_ENTRADA_MACRO'].notna()
df_maestro['PERIODO_A_PREDECIR'] = (df_maestro['PERIODO'] > df_maestro['PERIODO_RETIRO'])

# Calcular meses desde retiro
def calcular_meses_retiro(grupo):
    retiro_periodo = grupo['PERIODO_RETIRO'].iloc[0]
    if pd.notna(retiro_periodo):
        retiro_fecha = grupo[grupo['PERIODO'] == retiro_periodo]['FECHA']
        if len(retiro_fecha) > 0:
            meses = ((grupo['FECHA'] - retiro_fecha.iloc[0]).dt.days / 30).round(0)
            return meses
    return pd.Series([np.nan] * len(grupo), index=grupo.index)

df_maestro['MESES_DESDE_RETIRO'] = df_maestro.groupby('VALVULA', group_keys=False).apply(calcular_meses_retiro).values

print(f"   ✓ Variables de identificación creadas")

# ============================================================================
# PASO 9: SELECCIONAR COLUMNAS FINALES
# ============================================================================
print("\n9. Seleccionando columnas finales...")

columnas_finales = [
    'VALVULA', 'PERIODO', 'AÑO', 'MES', 'FECHA',
    # Variables objetivo
    'VOLUMEN_ENTRADA_FINAL', 'VOLUMEN_SALIDA_FINAL',
    'PERDIDAS_FINAL', 'INDICE_PERDIDAS_FINAL',
    # Variables predictoras
    'PRESION_FINAL', 'TEMPERATURA_FINAL', 'KPT_FINAL',
    'NUM_USUARIOS', 'NUM_REGISTROS',
    # Variables de control
    'TIENE_MACROMEDIDOR', 'PERIODO_A_PREDECIR', 'MESES_DESDE_RETIRO',
    # Variables originales (para análisis)
    'VOLUMEN_ENTRADA_MACRO', 'VOLUMEN_SALIDA_USUARIOS',
    'PRESION_MACRO', 'TEMPERATURA_MACRO', 'KPT_MACRO'
]

# Filtrar solo columnas que existen
columnas_disponibles = [col for col in columnas_finales if col in df_maestro.columns]
df_final = df_maestro[columnas_disponibles].copy()

print(f"   ✓ Dataset final: {df_final.shape}")

# ============================================================================
# PASO 10: RESUMEN Y VALIDACIÓN
# ============================================================================
print("\n10. Resumen del dataset maestro:")

print("\n   Registros por válvula:")
print(df_final.groupby('VALVULA').size())

print("\n   Periodos con/sin macromedidor:")
resumen = df_final.groupby(['VALVULA', 'TIENE_MACROMEDIDOR']).size().unstack(fill_value=0)
resumen.columns = ['Sin_Macro', 'Con_Macro']
print(resumen)

print("\n   Rango temporal por válvula:")
rango_temp = df_final.groupby('VALVULA')['FECHA'].agg(['min', 'max', 'count'])
rango_temp.columns = ['Fecha_Inicio', 'Fecha_Fin', 'Total_Meses']
print(rango_temp)

print("\n   Periodos a predecir por válvula:")
pred = df_final[df_final['PERIODO_A_PREDECIR'] == True].groupby('VALVULA').size()
print(pred)

# ============================================================================
# PASO 11: GUARDAR RESULTADOS
# ============================================================================
print("\n11. Guardando resultados...")

# Guardar dataset completo
df_final.to_csv('Dataset_Maestro_Balances.csv', index=False, sep=';', decimal=',', encoding='latin-1')
print(f"   ✓ Dataset maestro completo: Dataset_Maestro_Balances.csv")

# Guardar solo periodos con macromedidor (para entrenamiento)
df_train = df_final[df_final['TIENE_MACROMEDIDOR'] == True].copy()
df_train.to_csv('Dataset_Train.csv', index=False, sep=';', decimal=',', encoding='latin-1')
print(f"   ✓ Dataset entrenamiento: Dataset_Train.csv ({df_train.shape})")

# Guardar solo periodos a predecir
df_pred = df_final[df_final['PERIODO_A_PREDECIR'] == True].copy()
df_pred.to_csv('Dataset_Prediccion.csv', index=False, sep=';', decimal=',', encoding='latin-1')
print(f"   ✓ Dataset predicción: Dataset_Prediccion.csv ({df_pred.shape})")

# Guardar resumen por válvula
resumen_valvula = df_final.groupby('VALVULA').agg({
    'FECHA': ['min', 'max'],
    'VOLUMEN_ENTRADA_FINAL': 'sum',
    'VOLUMEN_SALIDA_FINAL': 'sum',
    'INDICE_PERDIDAS_FINAL': 'mean',
    'TIENE_MACROMEDIDOR': 'sum',
    'PERIODO_A_PREDECIR': 'sum'
}).reset_index()
resumen_valvula.columns = ['_'.join(col).strip('_') for col in resumen_valvula.columns.values]
resumen_valvula.to_csv('Resumen_Valvulas.csv', index=False, sep=';', decimal=',', encoding='latin-1')
print(f"   ✓ Resumen: Resumen_Valvulas.csv")

print("\n" + "=" * 80)
print("PROCESO COMPLETADO ✓")
print("=" * 80)
print(f"\n📊 Dataset maestro creado con {df_final.shape[0]} registros y {df_final.shape[1]} columnas")
print(f"\n📝 Archivos generados:")
print(f"   • Dataset_Maestro_Balances.csv - Dataset completo")
print(f"   • Dataset_Train.csv - Solo períodos con macromedidor (entrenamiento)")
print(f"   • Dataset_Prediccion.csv - Solo períodos a predecir")
print(f"   • Resumen_Valvulas.csv - Resumen por válvula")
print(f"\n🎯 Variable objetivo a predecir: VOLUMEN_ENTRADA_FINAL")
print(f"   (en períodos donde PERIODO_A_PREDECIR = True)")

# Mostrar muestra
print("\n📋 Muestra del dataset (últimos 10 registros):")
print(df_final[['VALVULA', 'PERIODO', 'VOLUMEN_ENTRADA_FINAL', 'VOLUMEN_SALIDA_FINAL',
                'INDICE_PERDIDAS_FINAL', 'TIENE_MACROMEDIDOR', 'PERIODO_A_PREDECIR']].tail(10))

CREACIÓN DE DATASET MAESTRO DE BALANCES

1. Cargando datasets...
   ✓ Balances históricos: (26, 14)
   ✓ Datos entrada: (5, 11)
   ✓ Macromedición mensual: (32, 9)
   ✓ Usuarios por válvula: (72, 10)

2. Procesando fechas de retiro del macromedidor...

   Información de retiro por válvula:
     VALVULA FECHA_RETIRO PERIODO_RETIRO  CANTIDAD_PERIODOS_PRONOSTICO
0  VALVULA_1   2025-07-21         202507                           9.0
1  VALVULA_2   2024-12-02         202412                          10.0
2  VALVULA_3   2025-06-24         202506                           3.0
3  VALVULA_4   2024-11-22         202411                          11.0
4  VALVULA_5   2024-10-04         202410                          12.0

3. Preparando balances históricos...
   Ejemplo de periodos creados: ['202502', '202501', '202412', '202411', '202410']
   ✓ Balances históricos procesados: (26, 10)

4. Preparando macromedición mensual...
   ✓ Macromedición mensual procesada: (32, 7)

5. Preparando usuarios por vá

C:\Users\karly\AppData\Local\Temp\ipykernel_17652\3477150713.py:303: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_maestro['MESES_DESDE_RETIRO'] = df_maestro.groupby('VALVULA', group_keys=False).apply(calcular_meses_retiro).values


In [8]:
# Ver qué datos tenemos en el dataset de entrenamiento
df_train = pd.read_csv('Dataset_Train.csv', sep=';', encoding='latin-1')

print("=" * 80)
print("ANÁLISIS DEL DATASET DE ENTRENAMIENTO")
print("=" * 80)

print("\n1. Registros por válvula con macromedidor:")
print(df_train.groupby('VALVULA').size())

print("\n2. Valores disponibles de VOLUMEN_ENTRADA_FINAL:")
print(f"   Total registros: {len(df_train)}")
print(f"   Con volumen entrada: {df_train['VOLUMEN_ENTRADA_FINAL'].notna().sum()}")
print(f"   Sin volumen entrada: {df_train['VOLUMEN_ENTRADA_FINAL'].isna().sum()}")

print("\n3. Muestra de datos con volumen:")
print(df_train[df_train['VOLUMEN_ENTRADA_FINAL'].notna()][
    ['VALVULA', 'PERIODO', 'VOLUMEN_ENTRADA_FINAL', 'VOLUMEN_SALIDA_FINAL',
     'PERDIDAS_FINAL', 'INDICE_PERDIDAS_FINAL']
].head(15))

print("\n4. Estadísticas por válvula:")
stats = df_train.groupby('VALVULA').agg({
    'VOLUMEN_ENTRADA_FINAL': lambda x: f"{x.notna().sum()}/{len(x)}",
    'PERIODO': ['min', 'max']
})
print(stats)

ANÁLISIS DEL DATASET DE ENTRENAMIENTO

1. Registros por válvula con macromedidor:
VALVULA
VALVULA_1    7
VALVULA_2    8
VALVULA_3    7
VALVULA_4    6
VALVULA_5    4
dtype: int64

2. Valores disponibles de VOLUMEN_ENTRADA_FINAL:
   Total registros: 32
   Con volumen entrada: 32
   Sin volumen entrada: 0

3. Muestra de datos con volumen:
      VALVULA  PERIODO VOLUMEN_ENTRADA_FINAL VOLUMEN_SALIDA_FINAL  \
0   VALVULA_1   202407                  66,5                  NaN   
1   VALVULA_1   202408    405,53000000000003                  NaN   
2   VALVULA_1   202409                334,21              377,786   
3   VALVULA_1   202410                428,15              443,825   
4   VALVULA_1   202411                414,74              430,229   
5   VALVULA_1   202412                399,51              407,892   
6   VALVULA_1   202501                472,57              483,643   
7   VALVULA_2   202405    1346,8700000000001                  NaN   
8   VALVULA_2   202406               2246

In [9]:
# ============================================================================
# EDA: EXPLORATORY DATA ANALYSIS - ANÁLISIS EXPLORATORIO DE DATOS
# ============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import warnings
warnings.filterwarnings('ignore')

print("=" * 80)
print("EDA: ANÁLISIS EXPLORATORIO DE DATOS")
print("=" * 80)

# Cargar dataset maestro
if not os.path.exists('Dataset_Maestro_Balances.csv'):
    print("❌ ERROR: Dataset_Maestro_Balances.csv no encontrado")
    print("   Ejecuta primero la Celda 12 (Construcción del Dataset Maestro)")
    raise FileNotFoundError("Dataset_Maestro_Balances.csv no encontrado")

df = pd.read_csv('Dataset_Maestro_Balances.csv', sep=';', decimal=',', encoding='latin-1')

# Asegurar tipos numéricos
numeric_cols = ['VOLUMEN_ENTRADA_FINAL', 'VOLUMEN_SALIDA_FINAL', 'PERDIDAS_FINAL', 
                'INDICE_PERDIDAS_FINAL', 'PRESION_FINAL', 'TEMPERATURA_FINAL', 
                'KPT_FINAL', 'NUM_USUARIOS', 'NUM_REGISTROS']
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

df['FECHA'] = pd.to_datetime(df['FECHA'], errors='coerce')

print(f"\n📊 Dataset cargado: {df.shape[0]} registros, {df.shape[1]} columnas")

# Crear directorio para gráficos EDA
os.makedirs('eda', exist_ok=True)

# ===== 1. ESTADÍSTICAS DESCRIPTIVAS =====
print("\n1. ESTADÍSTICAS DESCRIPTIVAS")
print("-" * 80)

print("\nResumen estadístico de variables numéricas:")
stats = df[numeric_cols].describe()
print(stats.round(2))

# Guardar estadísticas
stats.to_csv('eda/Estadisticas_Descriptivas.csv', sep=';', decimal=',', encoding='latin-1')
print("\n✓ Estadísticas guardadas en: eda/Estadisticas_Descriptivas.csv")

# ===== 2. ANÁLISIS DE VALORES FALTANTES =====
print("\n2. ANÁLISIS DE VALORES FALTANTES")
print("-" * 80)

missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Variable': missing.index,
    'Valores_Faltantes': missing.values,
    'Porcentaje': missing_pct.values
}).sort_values('Valores_Faltantes', ascending=False)

missing_df = missing_df[missing_df['Valores_Faltantes'] > 0]
if len(missing_df) > 0:
    print("\nVariables con valores faltantes:")
    print(missing_df.to_string(index=False))
    
    # Visualización
    fig = px.bar(missing_df, x='Variable', y='Porcentaje', 
                 title='Porcentaje de Valores Faltantes por Variable',
                 labels={'Porcentaje': 'Porcentaje (%)'})
    fig.write_html('eda/Valores_Faltantes.html')
    print("✓ Gráfico guardado en: eda/Valores_Faltantes.html")
else:
    print("✓ No hay valores faltantes")

missing_df.to_csv('eda/Valores_Faltantes.csv', index=False, sep=';', decimal=',', encoding='latin-1')

# ===== 3. ANÁLISIS DE CORRELACIONES =====
print("\n3. ANÁLISIS DE CORRELACIONES")
print("-" * 80)

# Seleccionar solo variables numéricas para correlación
corr_cols = [c for c in numeric_cols if c in df.columns]
df_corr = df[corr_cols].corr()

print("\nMatriz de correlación:")
print(df_corr.round(3))

# Guardar matriz de correlación
df_corr.to_csv('eda/Matriz_Correlacion.csv', sep=';', decimal=',', encoding='latin-1')
print("\n✓ Matriz de correlación guardada en: eda/Matriz_Correlacion.csv")

# Correlaciones con variable objetivo
if 'VOLUMEN_ENTRADA_FINAL' in df_corr.columns:
    target_corr = df_corr['VOLUMEN_ENTRADA_FINAL'].sort_values(ascending=False)
    print("\nCorrelaciones con VOLUMEN_ENTRADA_FINAL (variable objetivo):")
    for var, corr in target_corr.items():
        if var != 'VOLUMEN_ENTRADA_FINAL':
            print(f"  {var}: {corr:.3f}")
            if abs(corr) > 0.7:
                print(f"    ✅ CORRELACIÓN FUERTE")
            elif abs(corr) > 0.5:
                print(f"    ⚠ Correlación moderada")
            elif abs(corr) > 0.3:
                print(f"    ⚠ Correlación débil")

# Visualización de matriz de correlación
fig = px.imshow(df_corr, 
                title='Matriz de Correlación entre Variables',
                color_continuous_scale='RdBu',
                aspect='auto',
                labels=dict(color="Correlación"))
fig.update_layout(height=600, width=800)
fig.write_html('eda/Matriz_Correlacion_Visual.html')
print("✓ Gráfico de correlación guardado en: eda/Matriz_Correlacion_Visual.html")

# ===== 4. DISTRIBUCIONES DE VARIABLES =====
print("\n4. ANÁLISIS DE DISTRIBUCIONES")
print("-" * 80)

# Distribuciones de variables clave
vars_principales = ['VOLUMEN_ENTRADA_FINAL', 'VOLUMEN_SALIDA_FINAL', 
                    'PERDIDAS_FINAL', 'INDICE_PERDIDAS_FINAL',
                    'PRESION_FINAL', 'TEMPERATURA_FINAL', 'KPT_FINAL']

for var in vars_principales:
    if var in df.columns:
        valores = df[var].dropna()
        if len(valores) > 0:
            print(f"\n{var}:")
            print(f"  Media: {valores.mean():.2f}")
            print(f"  Mediana: {valores.median():.2f}")
            print(f"  Std: {valores.std():.2f}")
            print(f"  Min: {valores.min():.2f}, Max: {valores.max():.2f}")
            print(f"  Coeficiente de variación: {(valores.std()/valores.mean()*100):.2f}%")
            
            # Skewness
            skew = valores.skew()
            if abs(skew) > 1:
                print(f"  ⚠ Distribución muy asimétrica (skew: {skew:.2f})")
            elif abs(skew) > 0.5:
                print(f"  ⚠ Distribución moderadamente asimétrica (skew: {skew:.2f})")
            else:
                print(f"  ✓ Distribución aproximadamente normal (skew: {skew:.2f})")

# Visualización de distribuciones
fig = make_subplots(
    rows=3, cols=3,
    subplot_titles=vars_principales[:9],
    vertical_spacing=0.12
)

for i, var in enumerate(vars_principales[:9]):
    if var in df.columns:
        row = (i // 3) + 1
        col = (i % 3) + 1
        valores = df[var].dropna()
        if len(valores) > 0:
            fig.add_trace(
                go.Histogram(x=valores, name=var, nbinsx=30),
                row=row, col=col
            )

fig.update_layout(height=900, title_text="Distribuciones de Variables Principales", showlegend=False)
fig.write_html('eda/Distribuciones_Variables.html')
print("\n✓ Gráficos de distribuciones guardados en: eda/Distribuciones_Variables.html")

# ===== 5. ANÁLISIS DE OUTLIERS =====
print("\n5. ANÁLISIS DE OUTLIERS")
print("-" * 80)

outliers_info = []

for var in vars_principales:
    if var in df.columns:
        valores = df[var].dropna()
        if len(valores) > 0:
            Q1 = valores.quantile(0.25)
            Q3 = valores.quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            outliers = valores[(valores < lower_bound) | (valores > upper_bound)]
            n_outliers = len(outliers)
            pct_outliers = (n_outliers / len(valores)) * 100
            
            outliers_info.append({
                'Variable': var,
                'Outliers': n_outliers,
                'Porcentaje': pct_outliers,
                'Lower_Bound': lower_bound,
                'Upper_Bound': upper_bound
            })
            
            if n_outliers > 0:
                print(f"\n{var}:")
                print(f"  Outliers detectados: {n_outliers} ({pct_outliers:.1f}%)")
                print(f"  Rango normal: [{lower_bound:.2f}, {upper_bound:.2f}]")
                if pct_outliers > 10:
                    print(f"  ⚠ ALTO porcentaje de outliers")

if len(outliers_info) > 0:
    df_outliers = pd.DataFrame(outliers_info)
    df_outliers.to_csv('eda/Analisis_Outliers.csv', index=False, sep=';', decimal=',', encoding='latin-1')
    print("\n✓ Análisis de outliers guardado en: eda/Analisis_Outliers.csv")

# ===== 6. ANÁLISIS POR VÁLVULA =====
print("\n6. ANÁLISIS POR VÁLVULA")
print("-" * 80)

if 'VALVULA' in df.columns:
    print("\nEstadísticas por válvula:")
    stats_valvula = df.groupby('VALVULA')[vars_principales].agg(['mean', 'std', 'count']).round(2)
    print(stats_valvula)
    
    stats_valvula.to_csv('eda/Estadisticas_Por_Valvula.csv', sep=';', decimal=',', encoding='latin-1')
    print("\n✓ Estadísticas por válvula guardadas en: eda/Estadisticas_Por_Valvula.csv")
    
    # Visualización comparativa
    if 'VOLUMEN_ENTRADA_FINAL' in df.columns:
        fig = px.box(df, x='VALVULA', y='VOLUMEN_ENTRADA_FINAL',
                    title='Distribución de Volumen de Entrada por Válvula')
        fig.write_html('eda/Boxplot_Entrada_Por_Valvula.html')
        print("✓ Boxplot guardado en: eda/Boxplot_Entrada_Por_Valvula.html")

# ===== 7. ANÁLISIS TEMPORAL =====
print("\n7. ANÁLISIS TEMPORAL")
print("-" * 80)

if 'FECHA' in df.columns and 'VOLUMEN_ENTRADA_FINAL' in df.columns:
    df_temp = df[df['FECHA'].notna()].copy()
    if len(df_temp) > 0:
        df_temp = df_temp.sort_values('FECHA')
        
        # Serie temporal de entrada
        fig = px.line(df_temp, x='FECHA', y='VOLUMEN_ENTRADA_FINAL', 
                     color='VALVULA' if 'VALVULA' in df_temp.columns else None,
                     title='Evolución Temporal del Volumen de Entrada',
                     labels={'VOLUMEN_ENTRADA_FINAL': 'Volumen Entrada (m³)', 'FECHA': 'Fecha'})
        fig.write_html('eda/Serie_Temporal_Entrada.html')
        print("✓ Serie temporal guardada en: eda/Serie_Temporal_Entrada.html")
        
        # Análisis de tendencias
        print("\nTendencias por válvula:")
        for v in df_temp['VALVULA'].unique() if 'VALVULA' in df_temp.columns else [None]:
            df_v = df_temp[df_temp['VALVULA'] == v] if v else df_temp
            if len(df_v) >= 3:
                valores = df_v['VOLUMEN_ENTRADA_FINAL'].dropna()
                if len(valores) >= 3:
                    # Tendencia simple (último - primero)
                    tendencia = valores.iloc[-1] - valores.iloc[0]
                    pct_cambio = (tendencia / valores.iloc[0]) * 100 if valores.iloc[0] != 0 else 0
                    if v:
                        print(f"  {v}: Cambio de {tendencia:.2f} ({pct_cambio:+.1f}%)")

# ===== 8. RELACIONES ENTRE VARIABLES =====
print("\n8. ANÁLISIS DE RELACIONES ENTRE VARIABLES")
print("-" * 80)

# Scatter plots de relaciones importantes
if 'VOLUMEN_ENTRADA_FINAL' in df.columns:
    relaciones = [
        ('VOLUMEN_SALIDA_FINAL', 'VOLUMEN_ENTRADA_FINAL'),
        ('PRESION_FINAL', 'VOLUMEN_ENTRADA_FINAL'),
        ('NUM_USUARIOS', 'VOLUMEN_ENTRADA_FINAL'),
        ('TEMPERATURA_FINAL', 'VOLUMEN_ENTRADA_FINAL')
    ]
    
    for var_x, var_y in relaciones:
        if var_x in df.columns and var_y in df.columns:
            df_plot = df[[var_x, var_y, 'VALVULA']].dropna() if 'VALVULA' in df.columns else df[[var_x, var_y]].dropna()
            if len(df_plot) > 0:
                fig = px.scatter(df_plot, x=var_x, y=var_y,
                               color='VALVULA' if 'VALVULA' in df_plot.columns else None,
                               title=f'Relación: {var_x} vs {var_y}',
                               trendline='ols' if len(df_plot) > 2 else None)
                filename = f"eda/Scatter_{var_x.replace('_', '')}_vs_{var_y.replace('_', '')}.html"
                fig.write_html(filename)
                print(f"✓ Gráfico guardado: {filename}")

# ===== 9. RESUMEN DE INSIGHTS =====
print("\n9. RESUMEN DE INSIGHTS Y RECOMENDACIONES")
print("-" * 80)

insights = []

# Insight 1: Correlaciones fuertes
if 'VOLUMEN_ENTRADA_FINAL' in df_corr.columns:
    strong_corr = df_corr['VOLUMEN_ENTRADA_FINAL'].abs().sort_values(ascending=False)
    strong_corr = strong_corr[(strong_corr > 0.5) & (strong_corr < 1.0)]
    if len(strong_corr) > 0:
        insights.append(f"✅ Variables fuertemente correlacionadas con VOLUMEN_ENTRADA_FINAL: {', '.join(strong_corr.index[:3].tolist())}")
    else:
        insights.append("⚠️ No hay variables fuertemente correlacionadas con VOLUMEN_ENTRADA_FINAL")

# Insight 2: Valores faltantes
if len(missing_df) > 0:
    max_missing = missing_df.iloc[0]
    insights.append(f"⚠️ Variable con más valores faltantes: {max_missing['Variable']} ({max_missing['Porcentaje']:.1f}%)")

# Insight 3: Outliers
if len(outliers_info) > 0:
    max_outliers = max(outliers_info, key=lambda x: x['Porcentaje'])
    if max_outliers['Porcentaje'] > 10:
        insights.append(f"⚠️ Variable con muchos outliers: {max_outliers['Variable']} ({max_outliers['Porcentaje']:.1f}%)")

# Insight 4: Variabilidad
if 'VOLUMEN_ENTRADA_FINAL' in df.columns:
    cv = (df['VOLUMEN_ENTRADA_FINAL'].std() / df['VOLUMEN_ENTRADA_FINAL'].mean()) * 100
    if cv > 50:
        insights.append(f"⚠️ Alta variabilidad en VOLUMEN_ENTRADA_FINAL (CV: {cv:.1f}%)")
    else:
        insights.append(f"✅ Variabilidad moderada en VOLUMEN_ENTRADA_FINAL (CV: {cv:.1f}%)")

for i, insight in enumerate(insights, 1):
    print(f"{i}. {insight}")

# Guardar insights
with open('eda/Insights_EDA.txt', 'w', encoding='utf-8') as f:
    f.write("INSIGHTS DEL ANÁLISIS EXPLORATORIO DE DATOS\n")
    f.write("=" * 80 + "\n\n")
    for i, insight in enumerate(insights, 1):
        f.write(f"{i}. {insight}\n")

print("\n✓ Insights guardados en: eda/Insights_EDA.txt")

print("\n" + "=" * 80)
print("EDA COMPLETADO")
print("=" * 80)
print("\n📁 Archivos generados en carpeta 'eda/':")
print("   • Estadisticas_Descriptivas.csv")
print("   • Valores_Faltantes.csv")
print("   • Matriz_Correlacion.csv")
print("   • Analisis_Outliers.csv")
print("   • Estadisticas_Por_Valvula.csv")
print("   • Insights_EDA.txt")
print("   • Gráficos HTML interactivos")



EDA: ANÁLISIS EXPLORATORIO DE DATOS

📊 Dataset cargado: 82 registros, 22 columnas

1. ESTADÍSTICAS DESCRIPTIVAS
--------------------------------------------------------------------------------

Resumen estadístico de variables numéricas:
       VOLUMEN_ENTRADA_FINAL  VOLUMEN_SALIDA_FINAL  PERDIDAS_FINAL  \
count                  36.00                 72.00           26.00   
mean                11349.36              12572.75         -649.05   
std                 12943.54              14049.49         1470.42   
min                    66.50                272.90        -5335.89   
25%                  1218.00               1699.95         -412.09   
50%                  3559.43               4760.62         -102.32   
75%                 27731.20              28893.38           -6.96   
max                 35938.86              39198.66          241.64   

       INDICE_PERDIDAS_FINAL  PRESION_FINAL  TEMPERATURA_FINAL  KPT_FINAL  \
count                  26.00          80.00           

## Entrenamiento y Pronóstico

En esta sección se entrenan modelos base por válvula y se generan pronósticos de `VOLUMEN_ENTRADA_FINAL` para los periodos con `PERIODO_A_PREDECIR = True` en `Dataset_Prediccion.csv`.

- Modelos: Prophet (serie por válvula) y LightGBM (features agregadas).
- Salidas: `Pronosticos.csv` y `Predicciones_Con_Balance.csv` con pérdidas e índice recalculados.
- Métricas: MAE, MAPE, RMSE por válvula en validación temporal simple.

In [10]:
# ============================================================================
# SERIALIZACIÓN DE MODELOS PARA PRODUCCIÓN (PKL/JOBLIB)
# ============================================================================
import pickle
import joblib
import os
import pandas as pd
import numpy as np
from prophet import Prophet
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor

print("=" * 80)
print("SERIALIZACIÓN DE MODELOS PARA PRODUCCIÓN")
print("=" * 80)

# Cargar datos necesarios
print("\nCargando datos...")
df_train = pd.read_csv('Dataset_Train.csv', sep=';', decimal=',', encoding='latin-1')
df_pred = pd.read_csv('Dataset_Prediccion.csv', sep=';', decimal=',', encoding='latin-1')

# Asegurar tipos
for col in ['VOLUMEN_ENTRADA_FINAL','PRESION_FINAL','TEMPERATURA_FINAL','KPT_FINAL','NUM_USUARIOS','NUM_REGISTROS']:
    if col in df_train.columns:
        df_train[col] = pd.to_numeric(df_train[col], errors='coerce')
df_train['FECHA'] = pd.to_datetime(df_train['FECHA'], errors='coerce')

# Función para crear features (misma que en entrenamiento)
def crear_features_mejoradas(df, hist_v=None):
    df = df.copy()
    feat_cols = []
    for c in ['PRESION_FINAL','TEMPERATURA_FINAL','KPT_FINAL','NUM_USUARIOS','NUM_REGISTROS','VOLUMEN_SALIDA_FINAL']:
        if c in df.columns:
            feat_cols.append(c)
    if 'FECHA' in df.columns:
        df['MES'] = df['FECHA'].dt.month
        df['AÑO'] = df['FECHA'].dt.year
        df['DIA_AÑO'] = df['FECHA'].dt.dayofyear
        feat_cols.extend(['MES', 'AÑO', 'DIA_AÑO'])
    if hist_v is not None and len(hist_v) > 0:
        hist_v = hist_v.sort_values('FECHA')
        if 'VOLUMEN_ENTRADA_FINAL' in hist_v.columns:
            valores = hist_v['VOLUMEN_ENTRADA_FINAL'].dropna().values
            if len(valores) > 0:
                df['LAG_1'] = valores[-1] if len(valores) >= 1 else np.nan
                df['MA_3'] = np.mean(valores[-3:]) if len(valores) >= 3 else np.mean(valores) if len(valores) > 0 else np.nan
                df['MA_6'] = np.mean(valores[-6:]) if len(valores) >= 6 else np.mean(valores) if len(valores) > 0 else np.nan
                feat_cols.extend(['LAG_1', 'MA_3', 'MA_6'])
    if 'PRESION_FINAL' in df.columns and 'TEMPERATURA_FINAL' in df.columns:
        df['PRESION_TEMP'] = df['PRESION_FINAL'] * df['TEMPERATURA_FINAL']
        feat_cols.append('PRESION_TEMP')
    if 'VOLUMEN_SALIDA_FINAL' in df.columns and 'NUM_USUARIOS' in df.columns:
        df['CONSUMO_POR_USUARIO'] = df['VOLUMEN_SALIDA_FINAL'] / (df['NUM_USUARIOS'] + 1)
        feat_cols.append('CONSUMO_POR_USUARIO')
    return df, feat_cols

# Reentrenar y guardar modelos
print("\nReentrenando y guardando modelos...")
valvulas = sorted(df_train['VALVULA'].dropna().unique())
modelos_entrenados = {}
metadata_modelos = {}

# Crear directorio para modelos
os.makedirs('modelos', exist_ok=True)

print(f"\nProcesando {len(valvulas)} válvulas...\n")

for v in valvulas:
    print(f"Procesando {v}...")
    hist_v = df_train[(df_train['VALVULA']==v) & (df_train['VOLUMEN_ENTRADA_FINAL'].notna())].copy()
    if hist_v.empty:
        continue
    
    hist_v = hist_v.sort_values('FECHA')
    n_hist = len(hist_v)
    modelos_v = {}
    metadata_v = {
        'valvula': v,
        'modelos_disponibles': [],
        'features_por_modelo': {},
        'fecha_entrenamiento': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    
    # Entrenar y guardar Prophet
    if n_hist >= 6:
        try:
            dfp = hist_v[['FECHA','VOLUMEN_ENTRADA_FINAL']].rename(columns={'FECHA':'ds','VOLUMEN_ENTRADA_FINAL':'y'})
            m_prophet = Prophet(seasonality_mode='multiplicative', yearly_seasonality=True, 
                              weekly_seasonality=False, daily_seasonality=False)
            m_prophet.fit(dfp)
            modelos_v['prophet'] = m_prophet
            metadata_v['modelos_disponibles'].append('prophet')
            print(f"  ✓ Prophet entrenado")
        except Exception as e:
            print(f"  ⚠ Prophet error: {str(e)[:50]}")
    
    # Preparar features
    hist_v_feat, feat_cols = crear_features_mejoradas(hist_v)
    feat_cols = [c for c in feat_cols if c in hist_v_feat.columns]
    feat_cols = [c for c in feat_cols if hist_v_feat[c].dtype in [np.float64, np.int64, np.float32, np.int32]]
    
    if len(feat_cols) > 0 and n_hist >= 6:
        X = hist_v_feat[feat_cols].fillna(method='ffill').fillna(0)
        y = hist_v['VOLUMEN_ENTRADA_FINAL'].values
        
        # LightGBM
        try:
            model_lgbm = LGBMRegressor(n_estimators=200, learning_rate=0.05, 
                                       subsample=0.9, colsample_bytree=0.8, 
                                       random_state=42, verbose=-1)
            model_lgbm.fit(X, y)
            modelos_v['lightgbm'] = model_lgbm
            modelos_v['lightgbm_features'] = feat_cols.copy()
            metadata_v['modelos_disponibles'].append('lightgbm')
            metadata_v['features_por_modelo']['lightgbm'] = feat_cols.copy()
            print(f"  ✓ LightGBM entrenado")
        except Exception as e:
            print(f"  ⚠ LightGBM error: {str(e)[:50]}")
        
        # Random Forest
        try:
            model_rf = RandomForestRegressor(n_estimators=100, max_depth=10, 
                                            min_samples_split=2, random_state=42, n_jobs=-1)
            model_rf.fit(X, y)
            modelos_v['randomforest'] = model_rf
            modelos_v['randomforest_features'] = feat_cols.copy()
            metadata_v['modelos_disponibles'].append('randomforest')
            metadata_v['features_por_modelo']['randomforest'] = feat_cols.copy()
            print(f"  ✓ RandomForest entrenado")
        except Exception as e:
            print(f"  ⚠ RandomForest error: {str(e)[:50]}")
        
        # CatBoost
        try:
            model_cat = CatBoostRegressor(iterations=100, learning_rate=0.05, 
                                         depth=6, random_state=42, verbose=False)
            model_cat.fit(X, y)
            modelos_v['catboost'] = model_cat
            modelos_v['catboost_features'] = feat_cols.copy()
            metadata_v['modelos_disponibles'].append('catboost')
            metadata_v['features_por_modelo']['catboost'] = feat_cols.copy()
            print(f"  ✓ CatBoost entrenado")
        except Exception as e:
            print(f"  ⚠ CatBoost error: {str(e)[:50]}")
    
    modelos_entrenados[v] = modelos_v
    metadata_modelos[v] = metadata_v

print(f"\nModelos entrenados para {len(modelos_entrenados)} válvulas\n")

for valvula, modelos_v in modelos_entrenados.items():
    print(f"Procesando {valvula}...")
    metadata_v = {
        'valvula': valvula,
        'modelos_disponibles': [],
        'features_por_modelo': {},
        'fecha_entrenamiento': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    
    # Serializar cada modelo
    for modelo_nombre, modelo_obj in modelos_v.items():
        if modelo_nombre.endswith('_features') or modelo_nombre.endswith('_seq_length'):
            continue  # Saltar metadata, se guarda aparte
            
        try:
            # Determinar nombre del archivo
            if modelo_nombre == 'prophet':
                filename = f'modelos/{valvula}_prophet.pkl'
                # Prophet se guarda mejor con pickle
                with open(filename, 'wb') as f:
                    pickle.dump(modelo_obj, f)
                metadata_v['modelos_disponibles'].append('prophet')
                if f'{valvula}_prophet_features' in modelos_v:
                    metadata_v['features_por_modelo']['prophet'] = modelos_v[f'{valvula}_prophet_features']
                print(f"  ✓ Prophet guardado: {filename}")
                
            elif modelo_nombre == 'lightgbm':
                filename = f'modelos/{valvula}_lightgbm.pkl'
                # LightGBM puede usar joblib o pickle
                joblib.dump(modelo_obj, filename)
                metadata_v['modelos_disponibles'].append('lightgbm')
                if f'{valvula}_lightgbm_features' in modelos_v:
                    metadata_v['features_por_modelo']['lightgbm'] = modelos_v[f'{valvula}_lightgbm_features']
                print(f"  ✓ LightGBM guardado: {filename}")
                
            elif modelo_nombre == 'randomforest':
                filename = f'modelos/{valvula}_randomforest.pkl'
                joblib.dump(modelo_obj, filename)
                metadata_v['modelos_disponibles'].append('randomforest')
                if f'{valvula}_randomforest_features' in modelos_v:
                    metadata_v['features_por_modelo']['randomforest'] = modelos_v[f'{valvula}_randomforest_features']
                print(f"  ✓ RandomForest guardado: {filename}")
                
            elif modelo_nombre == 'catboost':
                filename = f'modelos/{valvula}_catboost.pkl'
                # CatBoost tiene su propio método de guardado
                modelo_obj.save_model(f'modelos/{valvula}_catboost.cbm')
                # También guardar con joblib para compatibilidad
                joblib.dump(modelo_obj, filename)
                metadata_v['modelos_disponibles'].append('catboost')
                if f'{valvula}_catboost_features' in modelos_v:
                    metadata_v['features_por_modelo']['catboost'] = modelos_v[f'{valvula}_catboost_features']
                print(f"  ✓ CatBoost guardado: {filename} y .cbm")
                
            elif modelo_nombre == 'hybrid_prophet':
                filename = f'modelos/{valvula}_hybrid_prophet.pkl'
                with open(filename, 'wb') as f:
                    pickle.dump(modelo_obj, f)
                metadata_v['modelos_disponibles'].append('hybrid_prophet')
                print(f"  ✓ Hybrid Prophet guardado: {filename}")
                
            elif modelo_nombre == 'hybrid_lstm':
                filename = f'modelos/{valvula}_hybrid_lstm.h5'
                # Guardar modelo Keras/TensorFlow
                modelo_obj.save(filename)
                metadata_v['modelos_disponibles'].append('hybrid_lstm')
                if f'{valvula}_hybrid_seq_length' in modelos_v:
                    metadata_v['hybrid_seq_length'] = modelos_v[f'{valvula}_hybrid_seq_length']
                print(f"  ✓ Hybrid LSTM guardado: {filename}")
                
        except Exception as e:
            print(f"  ⚠ Error guardando {modelo_nombre}: {str(e)[:50]}")
    
    metadata_modelos[valvula] = metadata_v

# Guardar metadata de modelos
with open('modelos/metadata_modelos.pkl', 'wb') as f:
    pickle.dump(metadata_modelos, f)

# También guardar metadata en JSON legible (si es posible)
try:
    import json
    metadata_json = {}
    for v, meta in metadata_modelos.items():
        metadata_json[v] = {
            'valvula': meta['valvula'],
            'modelos_disponibles': meta['modelos_disponibles'],
            'features_por_modelo': {k: list(v) if isinstance(v, list) else v 
                                   for k, v in meta['features_por_modelo'].items()},
            'fecha_entrenamiento': meta['fecha_entrenamiento']
        }
    with open('modelos/metadata_modelos.json', 'w', encoding='utf-8') as f:
        json.dump(metadata_json, f, indent=2, ensure_ascii=False)
    print("\n✓ Metadata guardada en JSON: modelos/metadata_modelos.json")
except Exception as e:
    print(f"\n⚠ No se pudo guardar metadata en JSON: {e}")

print(f"\n✓ Metadata guardada en: modelos/metadata_modelos.pkl")

# Crear script de carga para uso en producción
script_carga = '''"""
Script para cargar y usar modelos en producción
"""
import pickle
import joblib
import pandas as pd
import numpy as np
from prophet import Prophet
from catboost import CatBoostRegressor

def cargar_modelos(valvula):
    """
    Carga todos los modelos entrenados para una válvula
    
    Args:
        valvula: Nombre de la válvula (ej: 'VALVULA_1')
    
    Returns:
        dict: Diccionario con modelos y metadata
    """
    # Cargar metadata
    with open('modelos/metadata_modelos.pkl', 'rb') as f:
        metadata = pickle.load(f)
    
    if valvula not in metadata:
        raise ValueError(f"Válvula {valvula} no encontrada en modelos")
    
    modelos = {}
    meta_v = metadata[valvula]
    
    # Cargar cada modelo disponible
    for modelo_nombre in meta_v['modelos_disponibles']:
        try:
            if modelo_nombre == 'prophet':
                with open(f'modelos/{valvula}_prophet.pkl', 'rb') as f:
                    modelos['prophet'] = pickle.load(f)
            elif modelo_nombre == 'lightgbm':
                modelos['lightgbm'] = joblib.load(f'modelos/{valvula}_lightgbm.pkl')
            elif modelo_nombre == 'randomforest':
                modelos['randomforest'] = joblib.load(f'modelos/{valvula}_randomforest.pkl')
            elif modelo_nombre == 'catboost':
                # Intentar cargar .cbm primero (más eficiente)
                try:
                    modelos['catboost'] = CatBoostRegressor()
                    modelos['catboost'].load_model(f'modelos/{valvula}_catboost.cbm')
                except:
                    modelos['catboost'] = joblib.load(f'modelos/{valvula}_catboost.pkl')
            elif modelo_nombre == 'hybrid_prophet':
                with open(f'modelos/{valvula}_hybrid_prophet.pkl', 'rb') as f:
                    modelos['hybrid_prophet'] = pickle.load(f)
            elif modelo_nombre == 'hybrid_lstm':
                from tensorflow.keras.models import load_model
                modelos['hybrid_lstm'] = load_model(f'modelos/{valvula}_hybrid_lstm.h5')
        except Exception as e:
            print(f"⚠ Error cargando {modelo_nombre}: {e}")
    
    return {
        'modelos': modelos,
        'metadata': meta_v
    }

def predecir_entrada(valvula, features_dict, fecha=None):
    """
    Hace predicción usando el ensemble de modelos
    
    Args:
        valvula: Nombre de la válvula
        features_dict: Diccionario con features necesarias
        fecha: Fecha para predicción (requerida para Prophet)
    
    Returns:
        float: Predicción de volumen de entrada
    """
    modelos_data = cargar_modelos(valvula)
    modelos = modelos_data['modelos']
    metadata = modelos_data['metadata']
    
    predicciones = []
    pesos = []
    
    # Prophet
    if 'prophet' in modelos and fecha is not None:
        try:
            future = pd.DataFrame({'ds': [pd.to_datetime(fecha)]})
            pred = modelos['prophet'].predict(future)['yhat'].values[0]
            predicciones.append(pred)
            pesos.append(0.2)  # Peso por defecto
        except:
            pass
    
    # Modelos basados en features
    for modelo_nombre in ['lightgbm', 'randomforest', 'catboost']:
        if modelo_nombre in modelos:
            try:
                feat_cols = metadata['features_por_modelo'].get(modelo_nombre, [])
                if feat_cols:
                    X = pd.DataFrame([features_dict])[feat_cols].fillna(0)
                    pred = modelos[modelo_nombre].predict(X)[0]
                    predicciones.append(pred)
                    # Peso basado en modelo (ajustar según métricas)
                    pesos_default = {'lightgbm': 0.25, 'randomforest': 0.25, 'catboost': 0.3}
                    pesos.append(pesos_default.get(modelo_nombre, 0.2))
            except Exception as e:
                print(f"⚠ Error en predicción {modelo_nombre}: {e}")
    
    # Ensemble
    if len(predicciones) > 0:
        pesos = np.array(pesos) / np.sum(pesos)
        return np.average(predicciones, weights=pesos)
    else:
        return None

# Ejemplo de uso:
# modelos_data = cargar_modelos('VALVULA_1')
# pred = predecir_entrada('VALVULA_1', {'PRESION_FINAL': 10.5, 'TEMPERATURA_FINAL': 25.0, ...}, fecha='2025-08-01')
'''

with open('modelos/cargar_modelos.py', 'w', encoding='utf-8') as f:
    f.write(script_carga)

print("✓ Script de carga creado: modelos/cargar_modelos.py")

# Resumen
print("\n" + "=" * 80)
print("RESUMEN DE SERIALIZACIÓN")
print("=" * 80)
print(f"\nModelos serializados para {len(metadata_modelos)} válvulas:")
for v, meta in metadata_modelos.items():
    print(f"\n  {v}:")
    print(f"    Modelos: {', '.join(meta['modelos_disponibles'])}")
    print(f"    Archivos generados:")
    for modelo in meta['modelos_disponibles']:
        if modelo == 'catboost':
            print(f"      - modelos/{v}_catboost.pkl")
            print(f"      - modelos/{v}_catboost.cbm")
        elif modelo == 'hybrid_lstm':
            print(f"      - modelos/{v}_hybrid_lstm.h5")
        else:
            print(f"      - modelos/{v}_{modelo}.pkl")

print("\n✓ Archivos generados:")
print("   • modelos/metadata_modelos.pkl - Metadata de modelos")
print("   • modelos/metadata_modelos.json - Metadata en JSON")
print("   • modelos/cargar_modelos.py - Script para cargar modelos")
print("   • modelos/{VALVULA}_{MODELO}.pkl - Modelos serializados")

print("\n" + "=" * 80)
print("SERIALIZACIÓN COMPLETADA")
print("=" * 80)
print("\n💡 Para usar en producción:")
print("   1. Copiar carpeta 'modelos/' al servidor/frontend")
print("   2. Instalar dependencias: pandas, numpy, prophet, lightgbm, scikit-learn, catboost")
print("   3. Usar script: from modelos.cargar_modelos import cargar_modelos, predecir_entrada")



SERIALIZACIÓN DE MODELOS PARA PRODUCCIÓN

Cargando datos...

Reentrenando y guardando modelos...

Procesando 5 válvulas...

Procesando VALVULA_1...


23:29:35 - cmdstanpy - INFO - Chain [1] start processing
23:29:38 - cmdstanpy - INFO - Chain [1] done processing


  ✓ Prophet entrenado
  ✓ LightGBM entrenado
  ✓ RandomForest entrenado


23:29:40 - cmdstanpy - INFO - Chain [1] start processing


  ✓ CatBoost entrenado
Procesando VALVULA_2...


...

  ✓ Prophet entrenado
  ✓ LightGBM entrenado
  ✓ RandomForest entrenado


23:29:46 - cmdstanpy - INFO - Chain [1] start processing


  ✓ CatBoost entrenado
Procesando VALVULA_4...


23:29:48 - cmdstanpy - INFO - Chain [1] done processing


  ✓ Prophet entrenado
  ✓ LightGBM entrenado
  ✓ RandomForest entrenado
  ✓ CatBoost entrenado
Procesando VALVULA_5...

Modelos entrenados para 5 válvulas

Procesando VALVULA_1...
  ✓ Prophet guardado: modelos/VALVULA_1_prophet.pkl
  ✓ LightGBM guardado: modelos/VALVULA_1_lightgbm.pkl
  ✓ RandomForest guardado: modelos/VALVULA_1_randomforest.pkl
  ✓ CatBoost guardado: modelos/VALVULA_1_catboost.pkl y .cbm
Procesando VALVULA_2...
  ✓ Prophet guardado: modelos/VALVULA_2_prophet.pkl
  ✓ LightGBM guardado: modelos/VALVULA_2_lightgbm.pkl
  ✓ RandomForest guardado: modelos/VALVULA_2_randomforest.pkl
  ✓ CatBoost guardado: modelos/VALVULA_2_catboost.pkl y .cbm
Procesando VALVULA_3...
  ✓ Prophet guardado: modelos/VALVULA_3_prophet.pkl
  ✓ LightGBM guardado: modelos/VALVULA_3_lightgbm.pkl
  ✓ RandomForest guardado: modelos/VALVULA_3_randomforest.pkl
  ✓ CatBoost guardado: modelos/VALVULA_3_catboost.pkl y .cbm
Procesando VALVULA_4...
  ✓ Prophet guardado: modelos/VALVULA_4_prophet.pkl
  ✓ Light

In [11]:
import pandas as pd
import numpy as np
from prophet import Prophet
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

# Intentar importar TensorFlow/Keras para LSTM (opcional)
try:
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout
    from tensorflow.keras.optimizers import Adam
    HAS_LSTM = True
except:
    HAS_LSTM = False
    print("⚠ TensorFlow no disponible, LSTM deshabilitado")

print("=" * 80)
print("ENTRENAMIENTO MEJORADO: MÚLTIPLES MODELOS POR VÁLVULA")
print("=" * 80)
print("Modelos: Prophet, LightGBM, Random Forest, CatBoost, Prophet+LSTM (híbrido)")

# Cargar datasets generados (parsing con decimal=',')
df_train = pd.read_csv('Dataset_Train.csv', sep=';', decimal=',', encoding='latin-1')
df_pred = pd.read_csv('Dataset_Prediccion.csv', sep=';', decimal=',', encoding='latin-1')

# Asegurar tipos
for col in ['VOLUMEN_ENTRADA_FINAL','VOLUMEN_SALIDA_FINAL','PRESION_FINAL','TEMPERATURA_FINAL','KPT_FINAL','NUM_USUARIOS','NUM_REGISTROS']:
    if col in df_train.columns:
        df_train[col] = pd.to_numeric(df_train[col], errors='coerce')
    if col in df_pred.columns:
        df_pred[col] = pd.to_numeric(df_pred[col], errors='coerce')
df_train['FECHA'] = pd.to_datetime(df_train['FECHA'], errors='coerce')
df_pred['FECHA'] = pd.to_datetime(df_pred['FECHA'], errors='coerce')

# Funciones auxiliares
def mase(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    if len(y_true) < 2:
        return np.nan
    denom = np.mean(np.abs(np.diff(y_true)))
    return np.mean(np.abs(y_true - y_pred)) / denom if denom > 0 else np.nan

def evaluar(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / np.where(y_true==0, np.nan, y_true))) * 100
    return mae, rmse, mape, mase(y_true, y_pred)

# Fallback ingenuo
def naive_forecast(hist_series, pred_index):
    hist_series = pd.Series(hist_series).dropna()
    if hist_series.empty:
        return pd.Series([np.nan] * len(pred_index), index=pred_index)
    if len(hist_series) >= 3:
        base = hist_series.tail(3).mean()
    else:
        base = hist_series.mean()
    return pd.Series([base] * len(pred_index), index=pred_index)

# Función para crear features mejoradas
def crear_features_mejoradas(df, hist_v=None):
    """Crea features adicionales incluyendo lags y estadísticas"""
    df = df.copy()
    
    # Features básicas
    feat_cols = []
    for c in ['PRESION_FINAL','TEMPERATURA_FINAL','KPT_FINAL','NUM_USUARIOS','NUM_REGISTROS','VOLUMEN_SALIDA_FINAL']:
        if c in df.columns:
            feat_cols.append(c)
    
    # Features temporales
    if 'FECHA' in df.columns:
        df['MES'] = df['FECHA'].dt.month
        df['AÑO'] = df['FECHA'].dt.year
        df['DIA_AÑO'] = df['FECHA'].dt.dayofyear
        feat_cols.extend(['MES', 'AÑO', 'DIA_AÑO'])
    
    # Lags y medias móviles (si hay histórico)
    if hist_v is not None and len(hist_v) > 0:
        hist_v = hist_v.sort_values('FECHA')
        if 'VOLUMEN_ENTRADA_FINAL' in hist_v.columns:
            valores = hist_v['VOLUMEN_ENTRADA_FINAL'].dropna().values
            if len(valores) > 0:
                # Último valor
                df['LAG_1'] = valores[-1] if len(valores) >= 1 else np.nan
                # Media de últimos 3
                df['MA_3'] = np.mean(valores[-3:]) if len(valores) >= 3 else np.mean(valores) if len(valores) > 0 else np.nan
                # Media de últimos 6
                df['MA_6'] = np.mean(valores[-6:]) if len(valores) >= 6 else np.mean(valores) if len(valores) > 0 else np.nan
                feat_cols.extend(['LAG_1', 'MA_3', 'MA_6'])
    
    # Features de interacción
    if 'PRESION_FINAL' in df.columns and 'TEMPERATURA_FINAL' in df.columns:
        df['PRESION_TEMP'] = df['PRESION_FINAL'] * df['TEMPERATURA_FINAL']
        feat_cols.append('PRESION_TEMP')
    
    if 'VOLUMEN_SALIDA_FINAL' in df.columns and 'NUM_USUARIOS' in df.columns:
        df['CONSUMO_POR_USUARIO'] = df['VOLUMEN_SALIDA_FINAL'] / (df['NUM_USUARIOS'] + 1)
        feat_cols.append('CONSUMO_POR_USUARIO')
    
    return df, feat_cols

# Función para LSTM simple
def entrenar_lstm_simple(X_seq, y_seq, n_features, epochs=50, verbose=0):
    """Entrena un LSTM simple para series temporales"""
    if not HAS_LSTM or len(X_seq) < 3:
        return None
    
    try:
        model = Sequential([
            LSTM(50, activation='relu', input_shape=(1, n_features), return_sequences=True),
            Dropout(0.2),
            LSTM(50, activation='relu'),
            Dropout(0.2),
            Dense(1)
        ])
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
        model.fit(X_seq, y_seq, epochs=epochs, verbose=verbose, batch_size=1)
        return model
    except:
        return None

# Entrenamiento de múltiples modelos por válvula
valvulas = sorted(df_train['VALVULA'].dropna().unique())
print(f"\nVálvulas para procesar: {len(valvulas)} -> {valvulas}\n")
resultados = []
pronosticos = []

for v in valvulas:
    hist_v = df_train[(df_train['VALVULA']==v) & (df_train['VOLUMEN_ENTRADA_FINAL'].notna())].copy()
    pred_v = df_pred[df_pred['VALVULA']==v].copy()
    print(f"{'='*80}")
    print(f"Procesando {v}: hist={len(hist_v)}, pred={len(pred_v)}")
    print(f"{'='*80}")
    
    if hist_v.empty or pred_v.empty:
        print("⚠ Saltando por falta de datos")
        continue
    
    hist_v = hist_v.sort_values('FECHA')
    n_hist = len(hist_v)
    
    # Inicializar predicciones
    pred_v['PRED_ENTRADA_PROPHET'] = np.nan
    pred_v['PRED_ENTRADA_LGBM'] = np.nan
    pred_v['PRED_ENTRADA_RF'] = np.nan
    pred_v['PRED_ENTRADA_CATBOOST'] = np.nan
    pred_v['PRED_ENTRADA_LSTM'] = np.nan
    pred_v['PRED_ENTRADA_HYBRID'] = np.nan
    
    # ===== MODELO 1: PROPHET =====
    dfp = hist_v[['FECHA','VOLUMEN_ENTRADA_FINAL']].rename(columns={'FECHA':'ds','VOLUMEN_ENTRADA_FINAL':'y'})
    if n_hist >= 6:
        try:
            m = Prophet(seasonality_mode='multiplicative', yearly_seasonality=True, 
                       weekly_seasonality=False, daily_seasonality=False)
            m.fit(dfp)
            future = pd.DataFrame({'ds': pred_v['FECHA']})
            fc_prophet = m.predict(future)
            pred_v['PRED_ENTRADA_PROPHET'] = fc_prophet['yhat'].values
            print("✓ Prophet OK")
        except Exception as e:
            pred_v['PRED_ENTRADA_PROPHET'] = naive_forecast(hist_v['VOLUMEN_ENTRADA_FINAL'], pred_v.index).values
            print(f"⚠ Prophet fallback: {str(e)[:50]}")
    else:
        pred_v['PRED_ENTRADA_PROPHET'] = naive_forecast(hist_v['VOLUMEN_ENTRADA_FINAL'], pred_v.index).values
        print("⚠ Prophet: pocos datos, usando fallback")
    
    # ===== PREPARAR FEATURES MEJORADAS =====
    hist_v_feat, feat_cols = crear_features_mejoradas(hist_v)
    pred_v_feat, _ = crear_features_mejoradas(pred_v, hist_v)
    
    # Filtrar solo features numéricas disponibles
    feat_cols = [c for c in feat_cols if c in hist_v_feat.columns and c in pred_v_feat.columns]
    feat_cols = [c for c in feat_cols if hist_v_feat[c].dtype in [np.float64, np.int64, np.float32, np.int32]]
    
    if len(feat_cols) == 0:
        print("⚠ No hay features disponibles para modelos basados en features")
    else:
        print(f"✓ Features disponibles: {len(feat_cols)} -> {feat_cols[:5]}...")
        
        # Preparar datos
        X = hist_v_feat[feat_cols].fillna(method='ffill').fillna(0)
        y = hist_v['VOLUMEN_ENTRADA_FINAL'].values
        
        # Validación temporal
        if n_hist >= 6:
            split_idx = max(1, int(len(X)*0.8))
            split_idx = min(split_idx, len(X)-2)
            X_tr, X_te = X.iloc[:split_idx], X.iloc[split_idx:]
            y_tr, y_te = y[:split_idx], y[split_idx:]
            
            # ===== MODELO 2: LIGHTGBM =====
            if n_hist >= 6:
                try:
                    model_lgbm = LGBMRegressor(n_estimators=200, learning_rate=0.05, 
                                               subsample=0.9, colsample_bytree=0.8, 
                                               random_state=42, verbose=-1)
                    model_lgbm.fit(X_tr, y_tr)
                    if len(y_te) >= 2:
                        y_hat = model_lgbm.predict(X_te)
                        mae, rmse, mape, mase_v = evaluar(y_te, y_hat)
                        resultados.append({'VALVULA': v, 'MODELO':'LightGBM', 'MAE': mae, 'RMSE': rmse, 'MAPE': mape, 'MASE': mase_v, 'N_TEST': len(y_te)})
                        print(f"  LightGBM test -> MAE: {mae:.2f}, RMSE: {rmse:.2f}, MAPE: {mape:.2f}%")
                    model_lgbm.fit(X, y)
                    X_pred = pred_v_feat[feat_cols].fillna(method='ffill').fillna(0)
                    pred_v['PRED_ENTRADA_LGBM'] = model_lgbm.predict(X_pred)
                    print("✓ LightGBM OK")
                except Exception as e:
                    print(f"⚠ LightGBM error: {str(e)[:50]}")
            
            # ===== MODELO 3: RANDOM FOREST =====
            try:
                model_rf = RandomForestRegressor(n_estimators=100, max_depth=10, 
                                                min_samples_split=2, random_state=42, n_jobs=-1)
                model_rf.fit(X_tr, y_tr)
                if len(y_te) >= 2:
                    y_hat = model_rf.predict(X_te)
                    mae, rmse, mape, mase_v = evaluar(y_te, y_hat)
                    resultados.append({'VALVULA': v, 'MODELO':'RandomForest', 'MAE': mae, 'RMSE': rmse, 'MAPE': mape, 'MASE': mase_v, 'N_TEST': len(y_te)})
                    print(f"  RandomForest test -> MAE: {mae:.2f}, RMSE: {rmse:.2f}, MAPE: {mape:.2f}%")
                model_rf.fit(X, y)
                X_pred = pred_v_feat[feat_cols].fillna(method='ffill').fillna(0)
                pred_v['PRED_ENTRADA_RF'] = model_rf.predict(X_pred)
                print("✓ Random Forest OK")
            except Exception as e:
                print(f"⚠ Random Forest error: {str(e)[:50]}")
            
            # ===== MODELO 4: CATBOOST =====
            try:
                model_cat = CatBoostRegressor(iterations=100, learning_rate=0.05, 
                                             depth=6, random_state=42, verbose=False)
                model_cat.fit(X_tr, y_tr)
                if len(y_te) >= 2:
                    y_hat = model_cat.predict(X_te)
                    mae, rmse, mape, mase_v = evaluar(y_te, y_hat)
                    resultados.append({'VALVULA': v, 'MODELO':'CatBoost', 'MAE': mae, 'RMSE': rmse, 'MAPE': mape, 'MASE': mase_v, 'N_TEST': len(y_te)})
                    print(f"  CatBoost test -> MAE: {mae:.2f}, RMSE: {rmse:.2f}, MAPE: {mape:.2f}%")
                model_cat.fit(X, y)
                X_pred = pred_v_feat[feat_cols].fillna(method='ffill').fillna(0)
                pred_v['PRED_ENTRADA_CATBOOST'] = model_cat.predict(X_pred)
                print("✓ CatBoost OK")
            except Exception as e:
                print(f"⚠ CatBoost error: {str(e)[:50]}")
            
            # ===== MODELO 5: HÍBRIDO PROPHET + LSTM =====
            if HAS_LSTM and n_hist >= 8:
                try:
                    # Usar residuos de Prophet para entrenar LSTM
                    prophet_train = m.predict(dfp[['ds']])
                    residuos = dfp['y'].values - prophet_train['yhat'].values
                    
                    # Crear secuencias para LSTM
                    seq_length = min(3, len(residuos)-1)
                    if seq_length >= 2:
                        X_seq = []
                        y_seq = []
                        for i in range(len(residuos) - seq_length):
                            X_seq.append(residuos[i:i+seq_length].reshape(1, seq_length))
                            y_seq.append(residuos[i+seq_length])
                        
                        if len(X_seq) > 0:
                            X_seq = np.array(X_seq)
                            y_seq = np.array(y_seq)
                            
                            # Entrenar LSTM en residuos
                            lstm_model = entrenar_lstm_simple(X_seq, y_seq, seq_length, epochs=30, verbose=0)
                            
                            if lstm_model is not None:
                                # Predecir residuos futuros
                                last_seq = residuos[-seq_length:].reshape(1, 1, seq_length)
                                residuos_pred = []
                                for _ in range(len(pred_v)):
                                    pred_res = lstm_model.predict(last_seq, verbose=0)[0, 0]
                                    residuos_pred.append(pred_res)
                                    # Actualizar secuencia
                                    last_seq = np.append(last_seq[0, 0, 1:], pred_res).reshape(1, 1, seq_length)
                                
                                # Combinar Prophet + residuos LSTM
                                pred_v['PRED_ENTRADA_HYBRID'] = pred_v['PRED_ENTRADA_PROPHET'].values + np.array(residuos_pred)
                                print("✓ Prophet+LSTM Híbrido OK")
                except Exception as e:
                    print(f"⚠ Híbrido Prophet+LSTM error: {str(e)[:50]}")
    
    # ===== ENSEMBLE FINAL: Pesos basados en métricas reales (CORREGIDO) =====
    preds_disponibles = []
    pesos = []
    metricas_por_modelo = {}  # Guardar métricas para usar en pesos
    
    # Obtener métricas de esta válvula de los resultados
    metricas_v = [r for r in resultados if r['VALVULA'] == v]
    for m in metricas_v:
        metricas_por_modelo[m['MODELO']] = m
    
    # Mapeo de modelos a columnas
    modelo_col_map = {
        'CatBoost': 'PRED_ENTRADA_CATBOOST',
        'RandomForest': 'PRED_ENTRADA_RF',
        'LightGBM': 'PRED_ENTRADA_LGBM',
        'Prophet': 'PRED_ENTRADA_PROPHET'
    }
    
    # ESTRATEGIA MEJORADA: Priorizar modelos con métricas, Prophet solo como respaldo
    modelos_con_metricas = []
    modelos_sin_metricas = []
    
    # Separar modelos con y sin métricas
    for modelo, col in modelo_col_map.items():
        if pred_v[col].notna().any():
            if modelo in metricas_por_modelo:
                modelos_con_metricas.append((modelo, col))
            else:
                modelos_sin_metricas.append((modelo, col))
    
    # Si hay modelos con métricas, usar solo esos (con pesos basados en MAE inverso)
    if len(modelos_con_metricas) > 0:
        # Calcular pesos basados en MAE inverso
        pesos_metricas = []
        for modelo, col in modelos_con_metricas:
            mae = metricas_por_modelo[modelo]['MAE']
            # Peso = 1/MAE (normalizado después)
            peso = 1.0 / (mae + 1e-6)
            preds_disponibles.append(col)
            pesos_metricas.append(peso)
        
        # Normalizar pesos de modelos con métricas
        pesos_metricas = np.array(pesos_metricas)
        pesos_metricas = pesos_metricas / np.sum(pesos_metricas)
        pesos = pesos_metricas.tolist()
        
        # Solo agregar Prophet si no hay suficientes modelos con métricas (máximo 1 modelo sin métricas)
        if len(modelos_con_metricas) < 2 and len(modelos_sin_metricas) > 0:
            # Agregar Prophet con peso bajo (10% del total)
            prophet_col = None
            for modelo, col in modelos_sin_metricas:
                if modelo == 'Prophet':
                    prophet_col = col
                    break
            
            if prophet_col is not None:
                # Reducir pesos existentes y agregar Prophet
                factor_reduccion = 0.9
                pesos = [p * factor_reduccion for p in pesos]
                preds_disponibles.append(prophet_col)
                pesos.append(0.1)  # 10% para Prophet
                # Renormalizar
                pesos = np.array(pesos)
                pesos = pesos / np.sum(pesos)
                pesos = pesos.tolist()
    else:
        # Si no hay métricas, usar todos los modelos disponibles con pesos por defecto
        pesos_default = {'CatBoost': 0.35, 'RandomForest': 0.25, 'LightGBM': 0.25, 'Prophet': 0.15}
        for modelo, col in modelos_sin_metricas:
            preds_disponibles.append(col)
            pesos.append(pesos_default.get(modelo, 0.2))
        # Normalizar
        if len(pesos) > 0:
            pesos = np.array(pesos)
            pesos = pesos / np.sum(pesos)
            pesos = pesos.tolist()
    
    # Agregar híbrido si está disponible (solo si hay otros modelos)
    if pred_v['PRED_ENTRADA_HYBRID'].notna().any() and len(preds_disponibles) > 0:
        # Reducir pesos existentes y agregar híbrido con 10%
        factor_reduccion = 0.9
        pesos = [p * factor_reduccion for p in pesos]
        preds_disponibles.append('PRED_ENTRADA_HYBRID')
        pesos.append(0.1)
        # Renormalizar
        pesos = np.array(pesos)
        pesos = pesos / np.sum(pesos)
        pesos = pesos.tolist()
    
    if len(preds_disponibles) > 0:
        # Normalizar pesos finales
        pesos = np.array(pesos)
        pesos = pesos / np.sum(pesos)
        # Promedio ponderado
        pred_v['PRED_ENTRADA'] = sum(pred_v[col].values * peso for col, peso in zip(preds_disponibles, pesos))
        
        # Mostrar pesos usados
        pesos_str = ", ".join([f"{col.split('_')[-1]}: {p:.2%}" for col, p in zip(preds_disponibles, pesos)])
        print(f"✓ Ensemble: {len(preds_disponibles)} modelos (pesos: {pesos_str})")
    else:
        pred_v['PRED_ENTRADA'] = naive_forecast(hist_v['VOLUMEN_ENTRADA_FINAL'], pred_v.index).values
        print("⚠ Ensemble: usando fallback ingenuo")
    
    # Recalcular pérdidas e índice
    pred_v['PRED_SALIDA'] = pred_v.get('VOLUMEN_SALIDA_FINAL', np.nan)
    pred_v['PRED_PERDIDAS'] = pred_v['PRED_ENTRADA'] - pred_v['PRED_SALIDA']
    pred_v['PRED_INDICE_PERDIDAS'] = np.where(pred_v['PRED_ENTRADA']>0, (pred_v['PRED_PERDIDAS']/pred_v['PRED_ENTRADA'])*100, np.nan)
    
    # Guardar todas las predicciones
    cols_guardar = ['VALVULA','PERIODO','FECHA','PRED_ENTRADA_PROPHET','PRED_ENTRADA_LGBM',
                   'PRED_ENTRADA_RF','PRED_ENTRADA_CATBOOST','PRED_ENTRADA_LSTM',
                   'PRED_ENTRADA_HYBRID','PRED_ENTRADA','PRED_SALIDA','PRED_PERDIDAS','PRED_INDICE_PERDIDAS']
    cols_guardar = [c for c in cols_guardar if c in pred_v.columns]
    pronosticos.append(pred_v[cols_guardar])
    print(f"✓ Pronósticos agregados: {len(pred_v)} filas\n")

# Concatenar y guardar
print(f"\n{'='*80}")
print(f"RESUMEN FINAL")
print(f"{'='*80}")
print(f"Total bloques de pronósticos: {len(pronosticos)}")

if len(pronosticos)>0:
    df_fc = pd.concat(pronosticos, ignore_index=True)
    df_fc.to_csv('Pronosticos.csv', index=False, sep=';', decimal=',', encoding='latin-1')
    print(f"✓ Pronósticos guardados: Pronosticos.csv ({df_fc.shape})")
    
    # Mostrar resumen de predicciones por modelo
    print("\nPredicciones disponibles por modelo:")
    for col in ['PRED_ENTRADA_PROPHET','PRED_ENTRADA_LGBM','PRED_ENTRADA_RF',
               'PRED_ENTRADA_CATBOOST','PRED_ENTRADA_HYBRID']:
        if col in df_fc.columns:
            n_valid = df_fc[col].notna().sum()
            print(f"  {col}: {n_valid}/{len(df_fc)} ({100*n_valid/len(df_fc):.1f}%)")
else:
    print("⚠ No se generaron pronósticos (verificar datos)")

# Guardar métricas
if len(resultados)>0:
    df_metrics = pd.DataFrame(resultados)
    df_metrics.to_csv('Metrics.csv', index=False, sep=';', decimal=',', encoding='latin-1')
    print(f"\n✓ Métricas guardadas: Metrics.csv ({df_metrics.shape})")
    print("\nResumen de métricas por modelo:")
    print(df_metrics.groupby('MODELO')[['MAE','RMSE','MAPE']].mean().round(2))
    print("\nMejores modelos por válvula:")
    for v in df_metrics['VALVULA'].unique():
        df_v = df_metrics[df_metrics['VALVULA']==v]
        mejor = df_v.loc[df_v['MAE'].idxmin()]
        print(f"  {v}: {mejor['MODELO']} (MAE: {mejor['MAE']:.2f})")
else:
    print("⚠ No se calcularon métricas (datos insuficientes)")

23:29:49 - cmdstanpy - INFO - Chain [1] start processing


⚠ TensorFlow no disponible, LSTM deshabilitado
ENTRENAMIENTO MEJORADO: MÚLTIPLES MODELOS POR VÁLVULA
Modelos: Prophet, LightGBM, Random Forest, CatBoost, Prophet+LSTM (híbrido)

Válvulas para procesar: 5 -> ['VALVULA_1', 'VALVULA_2', 'VALVULA_3', 'VALVULA_4', 'VALVULA_5']

Procesando VALVULA_1: hist=7, pred=4


23:29:52 - cmdstanpy - INFO - Chain [1] done processing


✓ Prophet OK
✓ Features disponibles: 11 -> ['PRESION_FINAL', 'TEMPERATURA_FINAL', 'KPT_FINAL', 'NUM_USUARIOS', 'NUM_REGISTROS']...
  LightGBM test -> MAE: 106.21, RMSE: 112.32, MAPE: 23.82%
✓ LightGBM OK
  RandomForest test -> MAE: 122.28, RMSE: 153.28, MAPE: 26.45%
✓ Random Forest OK
  CatBoost test -> MAE: 71.41, RMSE: 94.80, MAPE: 15.29%
✓ CatBoost OK
✓ Ensemble: 3 modelos (pesos: CATBOOST: 44.32%, RF: 25.88%, LGBM: 29.80%)
✓ Pronósticos agregados: 4 filas

Procesando VALVULA_2: hist=8, pred=11


23:29:53 - cmdstanpy - INFO - Chain [1] start processing
23:29:57 - cmdstanpy - INFO - Chain [1] done processing


✓ Prophet OK
✓ Features disponibles: 11 -> ['PRESION_FINAL', 'TEMPERATURA_FINAL', 'KPT_FINAL', 'NUM_USUARIOS', 'NUM_REGISTROS']...
  LightGBM test -> MAE: 323.74, RMSE: 337.76, MAPE: 18.04%
✓ LightGBM OK
  RandomForest test -> MAE: 438.66, RMSE: 463.96, MAPE: 24.50%
✓ Random Forest OK
  CatBoost test -> MAE: 401.64, RMSE: 417.07, MAPE: 22.36%
✓ CatBoost OK
✓ Ensemble: 3 modelos (pesos: CATBOOST: 31.68%, RF: 29.01%, LGBM: 39.31%)
✓ Pronósticos agregados: 11 filas

Procesando VALVULA_3: hist=7, pred=5


23:29:57 - cmdstanpy - INFO - Chain [1] start processing
23:29:58 - cmdstanpy - INFO - Chain [1] done processing


✓ Prophet OK
✓ Features disponibles: 11 -> ['PRESION_FINAL', 'TEMPERATURA_FINAL', 'KPT_FINAL', 'NUM_USUARIOS', 'NUM_REGISTROS']...
  LightGBM test -> MAE: 4448.63, RMSE: 4457.67, MAPE: 15.71%
✓ LightGBM OK
  RandomForest test -> MAE: 1090.91, RMSE: 1125.07, MAPE: 3.87%
✓ Random Forest OK
  CatBoost test -> MAE: 2192.96, RMSE: 2258.65, MAPE: 7.73%
✓ CatBoost OK
✓ Ensemble: 3 modelos (pesos: CATBOOST: 28.55%, RF: 57.38%, LGBM: 14.07%)
✓ Pronósticos agregados: 5 filas

Procesando VALVULA_4: hist=6, pred=12


23:29:59 - cmdstanpy - INFO - Chain [1] start processing
23:30:01 - cmdstanpy - INFO - Chain [1] done processing


✓ Prophet OK
✓ Features disponibles: 11 -> ['PRESION_FINAL', 'TEMPERATURA_FINAL', 'KPT_FINAL', 'NUM_USUARIOS', 'NUM_REGISTROS']...
  LightGBM test -> MAE: 5758.28, RMSE: 6160.97, MAPE: 18.51%
✓ LightGBM OK
  RandomForest test -> MAE: 4739.57, RMSE: 5555.82, MAPE: 14.98%
✓ Random Forest OK
  CatBoost test -> MAE: 3602.33, RMSE: 4335.42, MAPE: 11.33%
✓ CatBoost OK
✓ Ensemble: 3 modelos (pesos: CATBOOST: 41.92%, RF: 31.86%, LGBM: 26.22%)
✓ Pronósticos agregados: 12 filas

Procesando VALVULA_5: hist=4, pred=13
⚠ Prophet: pocos datos, usando fallback
✓ Features disponibles: 11 -> ['PRESION_FINAL', 'TEMPERATURA_FINAL', 'KPT_FINAL', 'NUM_USUARIOS', 'NUM_REGISTROS']...
✓ Ensemble: 1 modelos (pesos: PROPHET: 100.00%)
✓ Pronósticos agregados: 13 filas


RESUMEN FINAL
Total bloques de pronósticos: 5
✓ Pronósticos guardados: Pronosticos.csv ((45, 13))

Predicciones disponibles por modelo:
  PRED_ENTRADA_PROPHET: 45/45 (100.0%)
  PRED_ENTRADA_LGBM: 32/45 (71.1%)
  PRED_ENTRADA_RF: 32/45 (71.1%)
  P

In [12]:
# ============================================================================
# ANÁLISIS DETALLADO DE RESULTADOS Y COMPARACIÓN DE MODELOS
# ============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("=" * 80)
print("ANÁLISIS DETALLADO DE RESULTADOS")
print("=" * 80)

# Cargar resultados
try:
    df_metrics = pd.read_csv('Metrics.csv', sep=';', decimal=',', encoding='latin-1')
    df_fc = pd.read_csv('Pronosticos.csv', sep=';', decimal=',', encoding='latin-1')
    df_train = pd.read_csv('Dataset_Train.csv', sep=';', decimal=',', encoding='latin-1')
except Exception as e:
    print(f"Error cargando archivos: {e}")
    raise

# Asegurar tipos numéricos
for col in df_metrics.select_dtypes(include=['object']).columns:
    if col not in ['VALVULA', 'MODELO']:
        df_metrics[col] = pd.to_numeric(df_metrics[col], errors='coerce')

print("\n1. RESUMEN DE MÉTRICAS POR MODELO (PROMEDIO)")
print("-" * 80)
if len(df_metrics) > 0:
    resumen = df_metrics.groupby('MODELO').agg({
        'MAE': ['mean', 'std', 'min', 'max'],
        'RMSE': ['mean', 'std'],
        'MAPE': ['mean', 'std'],
        'MASE': ['mean', 'std']
    }).round(2)
    print(resumen)
    
    print("\n2. RANKING DE MODELOS POR MÉTRICA")
    print("-" * 80)
    print("Por MAE (menor es mejor):")
    ranking_mae = df_metrics.groupby('MODELO')['MAE'].mean().sort_values()
    for i, (modelo, mae) in enumerate(ranking_mae.items(), 1):
        print(f"  {i}. {modelo}: MAE promedio = {mae:.2f}")
    
    print("\nPor MAPE (menor es mejor):")
    ranking_mape = df_metrics.groupby('MODELO')['MAPE'].mean().sort_values()
    for i, (modelo, mape) in enumerate(ranking_mape.items(), 1):
        print(f"  {i}. {modelo}: MAPE promedio = {mape:.2f}%")
    
    print("\n3. MEJOR MODELO POR VÁLVULA")
    print("-" * 80)
    for v in sorted(df_metrics['VALVULA'].unique()):
        df_v = df_metrics[df_metrics['VALVULA'] == v]
        mejor_mae = df_v.loc[df_v['MAE'].idxmin()]
        mejor_mape = df_v.loc[df_v['MAPE'].idxmin()]
        print(f"\n{v}:")
        print(f"  Mejor MAE: {mejor_mae['MODELO']} (MAE: {mejor_mae['MAE']:.2f}, MAPE: {mejor_mae['MAPE']:.2f}%)")
        if mejor_mae['MODELO'] != mejor_mape['MODELO']:
            print(f"  Mejor MAPE: {mejor_mape['MODELO']} (MAE: {mejor_mape['MAE']:.2f}, MAPE: {mejor_mape['MAPE']:.2f}%)")
    
    print("\n4. ANÁLISIS DE VARIABILIDAD")
    print("-" * 80)
    print("Desviación estándar de MAE por modelo (menor = más consistente):")
    std_mae = df_metrics.groupby('MODELO')['MAE'].std().sort_values()
    for modelo, std in std_mae.items():
        print(f"  {modelo}: σ = {std:.2f}")
    
    print("\n5. COMPARACIÓN DE PREDICCIONES")
    print("-" * 80)
    # Comparar predicciones de diferentes modelos
    pred_cols = [c for c in df_fc.columns if c.startswith('PRED_ENTRADA') and c != 'PRED_ENTRADA']
    
    if len(pred_cols) > 0:
        print(f"Modelos con predicciones: {len(pred_cols)}")
        print("\nEstadísticas de predicciones por modelo:")
        for col in pred_cols:
            valores = df_fc[col].dropna()
            if len(valores) > 0:
                modelo_nombre = col.replace('PRED_ENTRADA_', '')
                print(f"\n  {modelo_nombre}:")
                print(f"    Predicciones válidas: {len(valores)}/{len(df_fc)} ({100*len(valores)/len(df_fc):.1f}%)")
                print(f"    Media: {valores.mean():.2f}")
                print(f"    Mediana: {valores.median():.2f}")
                print(f"    Std: {valores.std():.2f}")
                print(f"    Min: {valores.min():.2f}, Max: {valores.max():.2f}")
        
        # Comparar con ensemble final
        if 'PRED_ENTRADA' in df_fc.columns:
            ensemble = df_fc['PRED_ENTRADA'].dropna()
            print(f"\n  Ensemble Final:")
            print(f"    Predicciones válidas: {len(ensemble)}/{len(df_fc)} ({100*len(ensemble)/len(df_fc):.1f}%)")
            print(f"    Media: {ensemble.mean():.2f}")
            print(f"    Mediana: {ensemble.median():.2f}")
            print(f"    Std: {ensemble.std():.2f}")
    
    print("\n6. RECOMENDACIONES")
    print("-" * 80)
    # Analizar qué modelo funciona mejor en general
    mejor_modelo_global = ranking_mae.index[0]
    print(f"Modelo recomendado (mejor MAE promedio): {mejor_modelo_global}")
    
    # Verificar consistencia
    modelos_consistentes = std_mae[std_mae < std_mae.median()].index.tolist()
    if modelos_consistentes:
        print(f"Modelos más consistentes (baja variabilidad): {', '.join(modelos_consistentes)}")
    
    # Verificar si hay válvulas problemáticas
    mae_por_valvula = df_metrics.groupby('VALVULA')['MAE'].mean().sort_values(ascending=False)
    if len(mae_por_valvula) > 0:
        peor_valvula = mae_por_valvula.index[-1]
        mejor_valvula = mae_por_valvula.index[0]
        print(f"\nVálvula con mejor rendimiento: {mejor_valvula} (MAE promedio: {mae_por_valvula[mejor_valvula]:.2f})")
        print(f"Válvula con peor rendimiento: {peor_valvula} (MAE promedio: {mae_por_valvula[peor_valvula]:.2f})")
        print(f"Ratio: {mae_por_valvula[peor_valvula] / mae_por_valvula[mejor_valvula]:.2f}x")
    
else:
    print("⚠ No hay métricas disponibles para análisis")

print("\n" + "=" * 80)
print("ANÁLISIS COMPLETADO")
print("=" * 80)

# Guardar resumen de análisis
if len(df_metrics) > 0:
    resumen_completo = df_metrics.groupby('MODELO').agg({
        'MAE': ['count', 'mean', 'std', 'min', 'max'],
        'RMSE': ['mean', 'std'],
        'MAPE': ['mean', 'std'],
        'MASE': ['mean']
    }).round(2)
    resumen_completo.to_csv('Resumen_Analisis_Modelos.csv', sep=';', decimal=',', encoding='latin-1')
    print("\n✓ Resumen guardado en: Resumen_Analisis_Modelos.csv")



ANÁLISIS DETALLADO DE RESULTADOS

1. RESUMEN DE MÉTRICAS POR MODELO (PROMEDIO)
--------------------------------------------------------------------------------
                  MAE                               RMSE            MAPE  \
                 mean      std     min      max     mean      std   mean   
MODELO                                                                     
CatBoost      1567.08  1646.13   71.41  3602.33  1776.49  1954.20  14.18   
LightGBM      2659.22  2873.93  106.21  5758.28  2767.18  3018.05  19.02   
RandomForest  1597.86  2132.95  122.28  4739.57  1824.53  2520.32  17.45   

                     MASE        
                std  mean   std  
MODELO                           
CatBoost       6.27  1.94  1.40  
LightGBM       3.43  3.07  3.18  
RandomForest  10.35  1.74  0.50  

2. RANKING DE MODELOS POR MÉTRICA
--------------------------------------------------------------------------------
Por MAE (menor es mejor):
  1. CatBoost: MAE promedio = 1567.08

In [13]:
# ============================================================================
# ENTREGABLE 1: TABLA DE BALANCES VIRTUALES POR PUNTO Y MES (m³) + ÍNDICE DE PÉRDIDAS
# ============================================================================
import pandas as pd
import numpy as np
import os

print("=" * 80)
print("ENTREGABLE 1: TABLA DE BALANCES VIRTUALES")
print("=" * 80)

# Verificar si existe Predicciones_Con_Balance.csv, si no, generarlo
if not os.path.exists('Predicciones_Con_Balance.csv'):
    print("⚠ Predicciones_Con_Balance.csv no encontrado. Generándolo...")
    
    # Verificar archivos necesarios
    archivos_necesarios = ['Dataset_Maestro_Balances.csv', 'Pronosticos.csv']
    faltantes = [f for f in archivos_necesarios if not os.path.exists(f)]
    
    if faltantes:
        print(f"❌ ERROR: Faltan archivos necesarios: {', '.join(faltantes)}")
        print("   Por favor, ejecuta primero:")
        print("   1. Celdas de preparación de datos (hasta Dataset_Maestro_Balances.csv)")
        print("   2. Celda 15 (Entrenamiento y Pronóstico)")
        raise FileNotFoundError(f"Archivos faltantes: {', '.join(faltantes)}")
    
    # Generar Predicciones_Con_Balance.csv
    try:
        df_maestro = pd.read_csv('Dataset_Maestro_Balances.csv', sep=';', decimal=',', encoding='latin-1')
        df_fc = pd.read_csv('Pronosticos.csv', sep=';', decimal=',', encoding='latin-1')
        
        # Normalizar tipos
        for col in ['VOLUMEN_ENTRADA_FINAL','VOLUMEN_SALIDA_FINAL','PERDIDAS_FINAL','INDICE_PERDIDAS_FINAL','PRED_ENTRADA','PRED_SALIDA','PRED_PERDIDAS','PRED_INDICE_PERDIDAS']:
            if col in df_maestro.columns:
                df_maestro[col] = pd.to_numeric(df_maestro[col], errors='coerce')
            if col in df_fc.columns:
                df_fc[col] = pd.to_numeric(df_fc[col], errors='coerce')
        
        df_maestro['FECHA'] = pd.to_datetime(df_maestro['FECHA'], errors='coerce')
        df_fc['FECHA'] = pd.to_datetime(df_fc['FECHA'], errors='coerce')
        
        # Unir por VALVULA + PERIODO
        df_out = df_maestro.merge(df_fc[['VALVULA','PERIODO','PRED_ENTRADA','PRED_SALIDA','PRED_PERDIDAS','PRED_INDICE_PERDIDAS']], 
                                 on=['VALVULA','PERIODO'], how='left')
        
        # Reemplazar entrada/salida en periodos a predecir
        mask_pred = df_out['PERIODO_A_PREDECIR'] == True
        df_out.loc[mask_pred, 'VOLUMEN_ENTRADA_FINAL'] = df_out.loc[mask_pred, 'PRED_ENTRADA']
        df_out.loc[mask_pred, 'VOLUMEN_SALIDA_FINAL'] = df_out.loc[mask_pred, 'PRED_SALIDA'].fillna(df_out.loc[mask_pred, 'VOLUMEN_SALIDA_FINAL'])
        
        # Recalcular pérdidas e índice para periodos a predecir
        df_out.loc[mask_pred, 'PERDIDAS_FINAL'] = df_out.loc[mask_pred, 'VOLUMEN_ENTRADA_FINAL'] - df_out.loc[mask_pred, 'VOLUMEN_SALIDA_FINAL']
        df_out.loc[mask_pred, 'INDICE_PERDIDAS_FINAL'] = np.where(df_out.loc[mask_pred, 'VOLUMEN_ENTRADA_FINAL']>0, 
                                                                  (df_out.loc[mask_pred, 'PERDIDAS_FINAL']/df_out.loc[mask_pred, 'VOLUMEN_ENTRADA_FINAL'])*100, np.nan)
        
        # Guardar resultado
        df_out.to_csv('Predicciones_Con_Balance.csv', index=False, sep=';', decimal=',', encoding='latin-1')
        print("✓ Predicciones_Con_Balance.csv generado exitosamente")
    except Exception as e:
        print(f"❌ ERROR al generar Predicciones_Con_Balance.csv: {e}")
        raise

# Cargar datos
df_balance = pd.read_csv('Predicciones_Con_Balance.csv', sep=';', decimal=',', encoding='latin-1')

# Asegurar tipos
for col in ['VOLUMEN_ENTRADA_FINAL','VOLUMEN_SALIDA_FINAL','PERDIDAS_FINAL','INDICE_PERDIDAS_FINAL']:
    if col in df_balance.columns:
        df_balance[col] = pd.to_numeric(df_balance[col], errors='coerce')

df_balance['FECHA'] = pd.to_datetime(df_balance['FECHA'], errors='coerce')
df_balance['AÑO'] = df_balance['FECHA'].dt.year
df_balance['MES'] = df_balance['FECHA'].dt.month

# Crear tabla de balances virtuales (formato entregable)
tabla_balances = df_balance[[
    'VALVULA', 'PERIODO', 'AÑO', 'MES', 'FECHA',
    'VOLUMEN_ENTRADA_FINAL', 'VOLUMEN_SALIDA_FINAL', 
    'PERDIDAS_FINAL', 'INDICE_PERDIDAS_FINAL',
    'PERIODO_A_PREDECIR'
]].copy()

# Renombrar columnas para el entregable
tabla_balances.rename(columns={
    'VALVULA': 'PUNTO',
    'VOLUMEN_ENTRADA_FINAL': 'ENTRADA_m3',
    'VOLUMEN_SALIDA_FINAL': 'SALIDA_m3',
    'PERDIDAS_FINAL': 'PERDIDAS_m3',
    'INDICE_PERDIDAS_FINAL': 'INDICE_PERDIDAS_%',
    'PERIODO_A_PREDECIR': 'ES_PRONOSTICO'
}, inplace=True)

# Ordenar por punto y fecha
tabla_balances = tabla_balances.sort_values(['PUNTO', 'FECHA']).reset_index(drop=True)

# Formatear valores
tabla_balances['ENTRADA_m3'] = tabla_balances['ENTRADA_m3'].round(2)
tabla_balances['SALIDA_m3'] = tabla_balances['SALIDA_m3'].round(2)
tabla_balances['PERDIDAS_m3'] = tabla_balances['PERDIDAS_m3'].round(2)
tabla_balances['INDICE_PERDIDAS_%'] = tabla_balances['INDICE_PERDIDAS_%'].round(2)

# Guardar tabla de balances virtuales
tabla_balances.to_csv('Tabla_Balances_Virtuales.csv', index=False, sep=';', decimal=',', encoding='latin-1')
print(f"✓ Tabla de balances virtuales guardada: Tabla_Balances_Virtuales.csv ({tabla_balances.shape})")

# Mostrar resumen
print("\nResumen por punto:")
resumen_puntos = tabla_balances.groupby('PUNTO').agg({
    'ENTRADA_m3': ['sum', 'mean', 'count'],
    'SALIDA_m3': ['sum', 'mean'],
    'PERDIDAS_m3': ['sum', 'mean'],
    'INDICE_PERDIDAS_%': 'mean',
    'ES_PRONOSTICO': 'sum'
}).round(2)
print(resumen_puntos)

# Mostrar muestra
print("\nMuestra de la tabla (primeros 10 registros):")
print(tabla_balances.head(10).to_string())

print("\n" + "=" * 80)
print("ENTREGABLE 1 COMPLETADO")
print("=" * 80)



ENTREGABLE 1: TABLA DE BALANCES VIRTUALES
⚠ Predicciones_Con_Balance.csv no encontrado. Generándolo...
✓ Predicciones_Con_Balance.csv generado exitosamente
✓ Tabla de balances virtuales guardada: Tabla_Balances_Virtuales.csv ((82, 10))

Resumen por punto:
          ENTRADA_m3                  SALIDA_m3           PERDIDAS_m3  \
                 sum      mean count        sum      mean         sum   
PUNTO                                                                   
VALVULA_1    4253.62    354.47    12    6977.91    465.19     -805.32   
VALVULA_2   39538.67   2080.98    19   40462.89   2380.17    -4517.94   
VALVULA_3  284403.94  25854.90    11  274879.28  30542.14   -24356.64   
VALVULA_4  463953.10  25775.17    18  509316.16  31832.26   -87051.04   
VALVULA_5   72883.73   4287.28    17   73601.67   4906.78    -5883.70   

                   INDICE_PERDIDAS_% ES_PRONOSTICO  
              mean              mean           sum  
PUNTO                                               


In [14]:
# ============================================================================
# ENTREGABLE 2: MÉTRICAS DE PERFORMANCE DEL MODELO Y BENCHMARK FRENTE A HISTÓRICO
# ============================================================================
import pandas as pd
import numpy as np
import os
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print("=" * 80)
print("ENTREGABLE 2: MÉTRICAS DE PERFORMANCE Y BENCHMARK")
print("=" * 80)

# Verificar y cargar archivos (con manejo de errores)
if not os.path.exists('Predicciones_Con_Balance.csv'):
    print("⚠ Predicciones_Con_Balance.csv no encontrado. Ejecuta primero la celda 17 o la celda que combina predicciones.")
    raise FileNotFoundError("Predicciones_Con_Balance.csv no encontrado")

df_balance = pd.read_csv('Predicciones_Con_Balance.csv', sep=';', decimal=',', encoding='latin-1')

if not os.path.exists('Dataset_Train.csv'):
    print("⚠ Dataset_Train.csv no encontrado. Algunas comparaciones no estarán disponibles.")
    df_train = pd.DataFrame()
else:
    df_train = pd.read_csv('Dataset_Train.csv', sep=';', decimal=',', encoding='latin-1')

if not os.path.exists('Metrics.csv'):
    print("⚠ Metrics.csv no encontrado. Las métricas de validación no estarán disponibles.")
    df_metrics = pd.DataFrame()
else:
    df_metrics = pd.read_csv('Metrics.csv', sep=';', decimal=',', encoding='latin-1')

# Asegurar tipos
for col in ['VOLUMEN_ENTRADA_FINAL','VOLUMEN_SALIDA_FINAL','PERDIDAS_FINAL','INDICE_PERDIDAS_FINAL']:
    if col in df_balance.columns:
        df_balance[col] = pd.to_numeric(df_balance[col], errors='coerce')
    if col in df_train.columns:
        df_train[col] = pd.to_numeric(df_train[col], errors='coerce')

# Separar histórico y pronóstico
df_historico = df_balance[df_balance['PERIODO_A_PREDECIR'] == False].copy()
df_pronostico = df_balance[df_balance['PERIODO_A_PREDECIR'] == True].copy()

# ===== 1. MÉTRICAS DEL MODELO (VALIDACIÓN) =====
print("\n1. MÉTRICAS DEL MODELO (VALIDACIÓN TEMPORAL)")
print("-" * 80)

if len(df_metrics) > 0:
    # Métricas por modelo
    metricas_modelo = df_metrics.groupby('MODELO').agg({
        'MAE': ['mean', 'std', 'min', 'max'],
        'RMSE': ['mean', 'std'],
        'MAPE': ['mean', 'std'],
        'MASE': ['mean', 'std']
    }).round(2)
    print("\nMétricas por modelo:")
    print(metricas_modelo)
    
    # Métricas por válvula
    print("\nMétricas por válvula:")
    metricas_valvula = df_metrics.groupby('VALVULA').agg({
        'MAE': ['mean', 'min'],
        'RMSE': 'mean',
        'MAPE': 'mean',
        'MASE': 'mean'
    }).round(2)
    print(metricas_valvula)
    
    # Mejor modelo por válvula
    print("\nMejor modelo por válvula:")
    for v in sorted(df_metrics['VALVULA'].unique()):
        df_v = df_metrics[df_metrics['VALVULA'] == v]
        mejor = df_v.loc[df_v['MAE'].idxmin()]
        print(f"  {v}: {mejor['MODELO']} - MAE: {mejor['MAE']:.2f}, MAPE: {mejor['MAPE']:.2f}%")
else:
    print("⚠ No hay métricas de validación disponibles")

# ===== 2. BENCHMARK: COMPARACIÓN HISTÓRICO vs PRONÓSTICO =====
print("\n2. BENCHMARK: HISTÓRICO vs PRONÓSTICO")
print("-" * 80)

if len(df_historico) > 0 and len(df_pronostico) > 0:
    # Estadísticas históricas
    stats_historico = df_historico.groupby('VALVULA').agg({
        'VOLUMEN_ENTRADA_FINAL': ['mean', 'std', 'min', 'max'],
        'VOLUMEN_SALIDA_FINAL': ['mean', 'std'],
        'PERDIDAS_FINAL': ['mean', 'std'],
        'INDICE_PERDIDAS_FINAL': ['mean', 'std']
    }).round(2)
    
    # Estadísticas de pronóstico
    stats_pronostico = df_pronostico.groupby('VALVULA').agg({
        'VOLUMEN_ENTRADA_FINAL': ['mean', 'std', 'min', 'max'],
        'VOLUMEN_SALIDA_FINAL': ['mean', 'std'],
        'PERDIDAS_FINAL': ['mean', 'std'],
        'INDICE_PERDIDAS_FINAL': ['mean', 'std']
    }).round(2)
    
    print("\nEstadísticas históricas (con macromedidor):")
    print(stats_historico)
    
    print("\nEstadísticas de pronóstico:")
    print(stats_pronostico)
    
    # Comparación de promedios
    print("\nComparación de promedios (Pronóstico vs Histórico):")
    comparacion = pd.DataFrame({
        'ENTRADA_HIST': stats_historico[('VOLUMEN_ENTRADA_FINAL', 'mean')],
        'ENTRADA_PRED': stats_pronostico[('VOLUMEN_ENTRADA_FINAL', 'mean')],
        'SALIDA_HIST': stats_historico[('VOLUMEN_SALIDA_FINAL', 'mean')],
        'SALIDA_PRED': stats_pronostico[('VOLUMEN_SALIDA_FINAL', 'mean')],
        'PERDIDAS_HIST': stats_historico[('PERDIDAS_FINAL', 'mean')],
        'PERDIDAS_PRED': stats_pronostico[('PERDIDAS_FINAL', 'mean')],
        'INDICE_HIST': stats_historico[('INDICE_PERDIDAS_FINAL', 'mean')],
        'INDICE_PRED': stats_pronostico[('INDICE_PERDIDAS_FINAL', 'mean')]
    })
    
    # Calcular diferencias y ratios
    comparacion['DIF_ENTRADA_%'] = ((comparacion['ENTRADA_PRED'] - comparacion['ENTRADA_HIST']) / comparacion['ENTRADA_HIST'] * 100).round(2)
    comparacion['DIF_SALIDA_%'] = ((comparacion['SALIDA_PRED'] - comparacion['SALIDA_HIST']) / comparacion['SALIDA_HIST'] * 100).round(2)
    comparacion['DIF_INDICE_%'] = (comparacion['INDICE_PRED'] - comparacion['INDICE_HIST']).round(2)
    
    print(comparacion)
    
    # Análisis de consistencia
    print("\nAnálisis de consistencia:")
    for v in comparacion.index:
        dif_entrada = comparacion.loc[v, 'DIF_ENTRADA_%']
        dif_indice = comparacion.loc[v, 'DIF_INDICE_%']
        print(f"\n{v}:")
        print(f"  Entrada: {dif_entrada:+.2f}% vs histórico")
        print(f"  Índice pérdidas: {dif_indice:+.2f} pp vs histórico")
        if abs(dif_entrada) > 20:
            print(f"  ⚠ Variación significativa en entrada (>20%)")
        if abs(dif_indice) > 5:
            print(f"  ⚠ Variación significativa en índice (>5pp)")

# ===== 3. MÉTRICAS DE CALIDAD DEL PRONÓSTICO =====
print("\n3. MÉTRICAS DE CALIDAD DEL PRONÓSTICO")
print("-" * 80)

# Si tenemos datos históricos, podemos evaluar la calidad del pronóstico
# comparando con tendencias históricas
if len(df_train) > 0:
    # Calcular tendencia histórica por válvula
    tendencias = {}
    for v in df_train['VALVULA'].unique():
        df_v = df_train[df_train['VALVULA'] == v].sort_values('FECHA')
        if len(df_v) >= 3:
            # Tendencia lineal simple
            valores = df_v['VOLUMEN_ENTRADA_FINAL'].dropna().values
            if len(valores) >= 3:
                # Media móvil de últimos 3
                tendencia = np.mean(valores[-3:])
                # Desviación estándar
                std = np.std(valores)
                tendencias[v] = {'media': tendencia, 'std': std}
    
    # Comparar pronósticos con tendencias
    if len(tendencias) > 0:
        print("\nComparación con tendencia histórica:")
        calidad_pronostico = []
        for v in df_pronostico['VALVULA'].unique():
            if v in tendencias:
                pred_v = df_pronostico[df_pronostico['VALVULA'] == v]
                entrada_pred = pred_v['VOLUMEN_ENTRADA_FINAL'].mean()
                entrada_hist = tendencias[v]['media']
                std_hist = tendencias[v]['std']
                
                # Z-score (cuántas desviaciones estándar está del histórico)
                z_score = (entrada_pred - entrada_hist) / (std_hist + 1e-6)
                
                calidad_pronostico.append({
                    'VALVULA': v,
                    'ENTRADA_PRED': entrada_pred,
                    'ENTRADA_HIST': entrada_hist,
                    'Z_SCORE': z_score,
                    'DENTRO_RANGO_2SIGMA': abs(z_score) <= 2
                })
        
        if len(calidad_pronostico) > 0:
            df_calidad = pd.DataFrame(calidad_pronostico)
            print(df_calidad.round(2))
            
            # Resumen
            dentro_rango = df_calidad['DENTRO_RANGO_2SIGMA'].sum()
            print(f"\nPronósticos dentro de rango histórico (±2σ): {dentro_rango}/{len(df_calidad)} ({100*dentro_rango/len(df_calidad):.1f}%)")

# ===== 4. GUARDAR REPORTE DE MÉTRICAS =====
print("\n4. GUARDANDO REPORTE DE MÉTRICAS")
print("-" * 80)

# Crear reporte consolidado
reporte_metricas = {
    'TIPO': [],
    'VALVULA': [],
    'METRICA': [],
    'VALOR': []
}

# Agregar métricas del modelo
if len(df_metrics) > 0:
    for _, row in df_metrics.iterrows():
        for metrica in ['MAE', 'RMSE', 'MAPE', 'MASE']:
            reporte_metricas['TIPO'].append('VALIDACION_MODELO')
            reporte_metricas['VALVULA'].append(row['VALVULA'])
            reporte_metricas['METRICA'].append(metrica)
            reporte_metricas['VALOR'].append(row[metrica])

# Agregar comparaciones histórico vs pronóstico
if len(comparacion) > 0:
    for v in comparacion.index:
        for metrica in ['DIF_ENTRADA_%', 'DIF_SALIDA_%', 'DIF_INDICE_%']:
            reporte_metricas['TIPO'].append('BENCHMARK_HISTORICO')
            reporte_metricas['VALVULA'].append(v)
            reporte_metricas['METRICA'].append(metrica)
            reporte_metricas['VALOR'].append(comparacion.loc[v, metrica])

df_reporte = pd.DataFrame(reporte_metricas)
if len(df_reporte) > 0:
    df_reporte.to_csv('Reporte_Metricas_Performance.csv', index=False, sep=';', decimal=',', encoding='latin-1')
    print(f"✓ Reporte de métricas guardado: Reporte_Metricas_Performance.csv ({df_reporte.shape})")

# Guardar comparación detallada
if len(comparacion) > 0:
    comparacion.to_csv('Benchmark_Historico_vs_Pronostico.csv', sep=';', decimal=',', encoding='latin-1')
    print(f"✓ Benchmark histórico guardado: Benchmark_Historico_vs_Pronostico.csv")

print("\n" + "=" * 80)
print("ENTREGABLE 2 COMPLETADO")
print("=" * 80)



ENTREGABLE 2: MÉTRICAS DE PERFORMANCE Y BENCHMARK

1. MÉTRICAS DEL MODELO (VALIDACIÓN TEMPORAL)
--------------------------------------------------------------------------------

Métricas por modelo:
                  MAE                               RMSE            MAPE  \
                 mean      std     min      max     mean      std   mean   
MODELO                                                                     
CatBoost      1567.08  1646.13   71.41  3602.33  1776.49  1954.20  14.18   
LightGBM      2659.22  2873.93  106.21  5758.28  2767.18  3018.05  19.02   
RandomForest  1597.86  2132.95  122.28  4739.57  1824.53  2520.32  17.45   

                     MASE        
                std  mean   std  
MODELO                           
CatBoost       6.27  1.94  1.40  
LightGBM       3.43  3.07  3.18  
RandomForest  10.35  1.74  0.50  

Métricas por válvula:
               MAE              RMSE   MAPE  MASE
              mean      min     mean   mean  mean
VALVULA          

In [15]:
# ============================================================================
# ENTREGABLE 4: DASHBOARD/REPORTE (GRÁFICOS, ALERTAS, TOP DESBALANCES)
# ============================================================================
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import os
from datetime import datetime

print("=" * 80)
print("ENTREGABLE 4: DASHBOARD Y REPORTE")
print("=" * 80)

# Verificar y cargar datos
if not os.path.exists('Predicciones_Con_Balance.csv'):
    print("❌ ERROR: Predicciones_Con_Balance.csv no encontrado.")
    print("   Por favor, ejecuta primero:")
    print("   1. Celda 15 (Entrenamiento y Pronóstico)")
    print("   2. Celda que combina predicciones con dataset maestro")
    print("   O ejecuta la Celda 17 que lo genera automáticamente")
    raise FileNotFoundError("Predicciones_Con_Balance.csv no encontrado")

df_balance = pd.read_csv('Predicciones_Con_Balance.csv', sep=';', decimal=',', encoding='latin-1')
df_metrics = pd.read_csv('Metrics.csv', sep=';', decimal=',', encoding='latin-1') if os.path.exists('Metrics.csv') else pd.DataFrame()

# Asegurar tipos
for col in ['VOLUMEN_ENTRADA_FINAL','VOLUMEN_SALIDA_FINAL','PERDIDAS_FINAL','INDICE_PERDIDAS_FINAL']:
    if col in df_balance.columns:
        df_balance[col] = pd.to_numeric(df_balance[col], errors='coerce')

df_balance['FECHA'] = pd.to_datetime(df_balance['FECHA'], errors='coerce')

# Crear directorio para gráficos
os.makedirs('dashboard', exist_ok=True)

# ===== 1. GRÁFICOS DE SERIES TEMPORALES POR VÁLVULA =====
print("\n1. Generando gráficos de series temporales...")

valvulas = sorted(df_balance['VALVULA'].dropna().unique())
graficas_html = []

for v in valvulas:
    df_v = df_balance[df_balance['VALVULA'] == v].sort_values('FECHA')
    
    # Crear figura con subplots
    fig = make_subplots(
        rows=3, cols=1,
        subplot_titles=(
            f'{v} - Entrada vs Salida (m³)',
            f'{v} - Pérdidas (m³)',
            f'{v} - Índice de Pérdidas (%)'
        ),
        vertical_spacing=0.1,
        row_heights=[0.4, 0.3, 0.3]
    )
    
    # Separar histórico y pronóstico
    df_hist = df_v[df_v['PERIODO_A_PREDECIR'] == False]
    df_pred = df_v[df_v['PERIODO_A_PREDECIR'] == True]
    
    # Gráfico 1: Entrada vs Salida
    if len(df_hist) > 0:
        fig.add_trace(go.Scatter(
            x=df_hist['FECHA'], y=df_hist['VOLUMEN_ENTRADA_FINAL'],
            mode='lines+markers', name='Entrada (Histórico)',
            line=dict(color='blue', width=2), marker=dict(size=6)
        ), row=1, col=1)
        fig.add_trace(go.Scatter(
            x=df_hist['FECHA'], y=df_hist['VOLUMEN_SALIDA_FINAL'],
            mode='lines+markers', name='Salida (Histórico)',
            line=dict(color='green', width=2), marker=dict(size=6)
        ), row=1, col=1)
    
    if len(df_pred) > 0:
        fig.add_trace(go.Scatter(
            x=df_pred['FECHA'], y=df_pred['VOLUMEN_ENTRADA_FINAL'],
            mode='lines+markers', name='Entrada (Pronóstico)',
            line=dict(color='blue', width=2, dash='dash'), marker=dict(size=6)
        ), row=1, col=1)
        fig.add_trace(go.Scatter(
            x=df_pred['FECHA'], y=df_pred['VOLUMEN_SALIDA_FINAL'],
            mode='lines+markers', name='Salida (Pronóstico)',
            line=dict(color='green', width=2, dash='dash'), marker=dict(size=6)
        ), row=1, col=1)
    
    # Gráfico 2: Pérdidas
    if len(df_hist) > 0:
        fig.add_trace(go.Scatter(
            x=df_hist['FECHA'], y=df_hist['PERDIDAS_FINAL'],
            mode='lines+markers', name='Pérdidas (Histórico)',
            line=dict(color='orange', width=2), marker=dict(size=6),
            showlegend=False
        ), row=2, col=1)
    
    if len(df_pred) > 0:
        fig.add_trace(go.Scatter(
            x=df_pred['FECHA'], y=df_pred['PERDIDAS_FINAL'],
            mode='lines+markers', name='Pérdidas (Pronóstico)',
            line=dict(color='orange', width=2, dash='dash'), marker=dict(size=6),
            showlegend=False
        ), row=2, col=1)
    
    # Línea de referencia en cero
    fig.add_hline(y=0, line_dash="dot", line_color="gray", row=2, col=1)
    
    # Gráfico 3: Índice de pérdidas
    if len(df_hist) > 0:
        fig.add_trace(go.Scatter(
            x=df_hist['FECHA'], y=df_hist['INDICE_PERDIDAS_FINAL'],
            mode='lines+markers', name='Índice (Histórico)',
            line=dict(color='red', width=2), marker=dict(size=6),
            showlegend=False
        ), row=3, col=1)
    
    if len(df_pred) > 0:
        fig.add_trace(go.Scatter(
            x=df_pred['FECHA'], y=df_pred['INDICE_PERDIDAS_FINAL'],
            mode='lines+markers', name='Índice (Pronóstico)',
            line=dict(color='red', width=2, dash='dash'), marker=dict(size=6),
            showlegend=False
        ), row=3, col=1)
    
    # Actualizar ejes
    fig.update_xaxes(title_text="Fecha", row=3, col=1)
    fig.update_yaxes(title_text="Volumen (m³)", row=1, col=1)
    fig.update_yaxes(title_text="Pérdidas (m³)", row=2, col=1)
    fig.update_yaxes(title_text="Índice (%)", row=3, col=1)
    
    # Actualizar layout
    fig.update_layout(
        height=900,
        title_text=f"Análisis Completo - {v}",
        hovermode='x unified'
    )
    
    # Guardar gráfico
    filename = f"dashboard/grafica_{v}.html"
    fig.write_html(filename)
    graficas_html.append(f"grafica_{v}.html")
    print(f"  ✓ {v}")

# ===== 2. ALERTAS POR PUNTO =====
print("\n2. Generando alertas por punto...")

alertas = []

# Definir umbrales de alerta
UMBRAL_INDICE_PERDIDAS_ALTO = 15  # %
UMBRAL_INDICE_PERDIDAS_CRITICO = 25  # %
UMBRAL_VARIACION_ENTRADA = 30  # %
UMBRAL_PREDICCION_NEGATIVA = True

for v in valvulas:
    df_v = df_balance[df_balance['VALVULA'] == v].sort_values('FECHA')
    df_pred = df_v[df_v['PERIODO_A_PREDECIR'] == True]
    df_hist = df_v[df_v['PERIODO_A_PREDECIR'] == False]
    
    if len(df_pred) == 0:
        continue
    
    # Calcular promedios
    indice_pred = df_pred['INDICE_PERDIDAS_FINAL'].mean()
    entrada_pred = df_pred['VOLUMEN_ENTRADA_FINAL'].mean()
    entrada_hist = df_hist['VOLUMEN_ENTRADA_FINAL'].mean() if len(df_hist) > 0 else None
    
    # Alertas
    nivel_alerta = 'OK'
    mensajes = []
    
    # Alerta 1: Índice de pérdidas alto
    if pd.notna(indice_pred):
        if indice_pred >= UMBRAL_INDICE_PERDIDAS_CRITICO:
            nivel_alerta = 'CRITICO'
            mensajes.append(f"Índice de pérdidas crítico: {indice_pred:.2f}%")
        elif indice_pred >= UMBRAL_INDICE_PERDIDAS_ALTO:
            nivel_alerta = 'ALTO' if nivel_alerta == 'OK' else nivel_alerta
            mensajes.append(f"Índice de pérdidas alto: {indice_pred:.2f}%")
    
    # Alerta 2: Variación significativa en entrada
    if entrada_hist is not None and pd.notna(entrada_pred) and pd.notna(entrada_hist):
        variacion = abs((entrada_pred - entrada_hist) / entrada_hist * 100)
        if variacion >= UMBRAL_VARIACION_ENTRADA:
            nivel_alerta = 'ALTO' if nivel_alerta == 'OK' else nivel_alerta
            mensajes.append(f"Variación significativa en entrada: {variacion:.1f}%")
    
    # Alerta 3: Pérdidas negativas (ganancias)
    perdidas_negativas = (df_pred['PERDIDAS_FINAL'] < 0).sum()
    if perdidas_negativas > 0:
        nivel_alerta = 'ALTO' if nivel_alerta == 'OK' else nivel_alerta
        mensajes.append(f"Pérdidas negativas en {perdidas_negativas} periodo(s)")
    
    # Alerta 4: Valores faltantes
    valores_faltantes = df_pred[['VOLUMEN_ENTRADA_FINAL', 'VOLUMEN_SALIDA_FINAL']].isna().sum().sum()
    if valores_faltantes > 0:
        nivel_alerta = 'ALTO' if nivel_alerta == 'OK' else nivel_alerta
        mensajes.append(f"Valores faltantes: {valores_faltantes}")
    
    if nivel_alerta != 'OK' or len(mensajes) > 0:
        alertas.append({
            'VALVULA': v,
            'NIVEL': nivel_alerta,
            'MENSAJES': ' | '.join(mensajes) if mensajes else 'Sin alertas',
            'INDICE_PERDIDAS_%': indice_pred,
            'ENTRADA_PROMEDIO': entrada_pred
        })

df_alertas = pd.DataFrame(alertas)
if len(df_alertas) > 0:
    df_alertas.to_csv('dashboard/Alertas_Puntos.csv', index=False, sep=';', decimal=',', encoding='latin-1')
    print(f"  ✓ {len(df_alertas)} alertas generadas")
    
    # Resumen de alertas
    print("\n  Resumen de alertas:")
    print(f"    CRÍTICAS: {len(df_alertas[df_alertas['NIVEL'] == 'CRITICO'])}")
    print(f"    ALTAS: {len(df_alertas[df_alertas['NIVEL'] == 'ALTO'])}")
    print(f"    OK: {len(df_alertas[df_alertas['NIVEL'] == 'OK'])}")
else:
    print("  ✓ Sin alertas")

# ===== 3. TOP DESBALANCES =====
print("\n3. Generando top desbalances...")

# Calcular desbalances por válvula (promedio del período de pronóstico)
desbalances = []
for v in valvulas:
    df_v = df_balance[df_balance['VALVULA'] == v]
    df_pred = df_v[df_v['PERIODO_A_PREDECIR'] == True]
    
    if len(df_pred) > 0:
        desbalances.append({
            'VALVULA': v,
            'PERDIDAS_PROMEDIO_m3': df_pred['PERDIDAS_FINAL'].mean(),
            'INDICE_PERDIDAS_%': df_pred['INDICE_PERDIDAS_FINAL'].mean(),
            'ENTRADA_PROMEDIO_m3': df_pred['VOLUMEN_ENTRADA_FINAL'].mean(),
            'SALIDA_PROMEDIO_m3': df_pred['VOLUMEN_SALIDA_FINAL'].mean(),
            'NUM_PERIODOS': len(df_pred)
        })

df_desbalances = pd.DataFrame(desbalances)
if len(df_desbalances) > 0:
    # Top por pérdidas absolutas
    df_desbalances['PERDIDAS_ABS'] = df_desbalances['PERDIDAS_PROMEDIO_m3'].abs()
    top_perdidas = df_desbalances.nlargest(10, 'PERDIDAS_ABS')
    
    # Top por índice de pérdidas
    top_indice = df_desbalances.nlargest(10, 'INDICE_PERDIDAS_%')
    
    # Guardar
    df_desbalances.to_csv('dashboard/Top_Desbalances.csv', index=False, sep=';', decimal=',', encoding='latin-1')
    top_perdidas.to_csv('dashboard/Top10_Perdidas_Absolutas.csv', index=False, sep=';', decimal=',', encoding='latin-1')
    top_indice.to_csv('dashboard/Top10_Indice_Perdidas.csv', index=False, sep=';', decimal=',', encoding='latin-1')
    
    print(f"  ✓ Top desbalances generado")
    print("\n  Top 5 por pérdidas absolutas:")
    for i, (_, row) in enumerate(top_perdidas.head(5).iterrows(), 1):
        print(f"    {i}. {row['VALVULA']}: {row['PERDIDAS_PROMEDIO_m3']:.2f} m³ ({row['INDICE_PERDIDAS_%']:.2f}%)")

# ===== 4. DASHBOARD HTML CONSOLIDADO =====
print("\n4. Generando dashboard HTML consolidado...")

html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Dashboard - Balances Virtuales</title>
    <meta charset="utf-8">
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 20px;
            background-color: #f5f5f5;
        }}
        .header {{
            background-color: #2c3e50;
            color: white;
            padding: 20px;
            border-radius: 5px;
            margin-bottom: 20px;
        }}
        .section {{
            background-color: white;
            padding: 20px;
            margin: 20px 0;
            border-radius: 5px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }}
        .alert {{
            padding: 10px;
            margin: 10px 0;
            border-radius: 5px;
        }}
        .alert-critico {{
            background-color: #ffebee;
            border-left: 4px solid #f44336;
        }}
        .alert-alto {{
            background-color: #fff3e0;
            border-left: 4px solid #ff9800;
        }}
        .alert-ok {{
            background-color: #e8f5e9;
            border-left: 4px solid #4caf50;
        }}
        table {{
            width: 100%;
            border-collapse: collapse;
            margin: 10px 0;
        }}
        th, td {{
            padding: 10px;
            text-align: left;
            border-bottom: 1px solid #ddd;
        }}
        th {{
            background-color: #2c3e50;
            color: white;
        }}
        iframe {{
            width: 100%;
            height: 900px;
            border: none;
            margin: 20px 0;
        }}
        .grafica-container {{
            margin: 30px 0;
        }}
    </style>
</head>
<body>
    <div class="header">
        <h1>📊 Dashboard - Balances Virtuales</h1>
        <p>Generado el: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
    </div>
    
    <div class="section">
        <h2>🚨 Alertas por Punto</h2>
"""

# Agregar alertas al HTML
if len(df_alertas) > 0:
    for _, alerta in df_alertas.iterrows():
        nivel_class = alerta['NIVEL'].lower()
        html_content += f"""
        <div class="alert alert-{nivel_class}">
            <strong>{alerta['VALVULA']}</strong> - {alerta['NIVEL']}<br>
            {alerta['MENSAJES']}
        </div>
        """
else:
    html_content += "<p>✅ No hay alertas activas</p>"

html_content += """
    </div>
    
    <div class="section">
        <h2>📈 Top 10 Desbalances (Pérdidas Absolutas)</h2>
        <table>
            <tr>
                <th>Válvula</th>
                <th>Pérdidas Promedio (m³)</th>
                <th>Índice de Pérdidas (%)</th>
                <th>Entrada Promedio (m³)</th>
                <th>Número de Períodos</th>
            </tr>
"""

# Agregar top desbalances
if len(df_desbalances) > 0:
    for _, row in top_perdidas.head(10).iterrows():
        html_content += f"""
            <tr>
                <td>{row['VALVULA']}</td>
                <td>{row['PERDIDAS_PROMEDIO_m3']:.2f}</td>
                <td>{row['INDICE_PERDIDAS_%']:.2f}</td>
                <td>{row['ENTRADA_PROMEDIO_m3']:.2f}</td>
                <td>{row['NUM_PERIODOS']}</td>
            </tr>
        """

html_content += """
        </table>
    </div>
    
    <div class="section">
        <h2>📊 Gráficos de Series Temporales</h2>
"""

# Agregar gráficos
for grafica in graficas_html:
    html_content += f"""
        <div class="grafica-container">
            <h3>{grafica.replace('grafica_', '').replace('.html', '')}</h3>
            <iframe src="{grafica}"></iframe>
        </div>
    """

html_content += """
    </div>
</body>
</html>
"""

# Guardar dashboard
with open('dashboard/Dashboard_Balances_Virtuales.html', 'w', encoding='utf-8') as f:
    f.write(html_content)

print(f"  ✓ Dashboard guardado: dashboard/Dashboard_Balances_Virtuales.html")

print("\n" + "=" * 80)
print("ENTREGABLE 4 COMPLETADO")
print("=" * 80)
print("\nArchivos generados:")
print("  • dashboard/Dashboard_Balances_Virtuales.html - Dashboard principal")
print("  • dashboard/grafica_*.html - Gráficos por válvula")
print("  • dashboard/Alertas_Puntos.csv - Alertas generadas")
print("  • dashboard/Top_Desbalances.csv - Análisis de desbalances")
print("  • dashboard/Top10_Perdidas_Absolutas.csv - Top 10 pérdidas")
print("  • dashboard/Top10_Indice_Perdidas.csv - Top 10 índice")



ENTREGABLE 4: DASHBOARD Y REPORTE

1. Generando gráficos de series temporales...
  ✓ VALVULA_1
  ✓ VALVULA_2
  ✓ VALVULA_3
  ✓ VALVULA_4
  ✓ VALVULA_5

2. Generando alertas por punto...
  ✓ 5 alertas generadas

  Resumen de alertas:
    CRÍTICAS: 0
    ALTAS: 5
    OK: 0

3. Generando top desbalances...
  ✓ Top desbalances generado

  Top 5 por pérdidas absolutas:
    1. VALVULA_4: -6693.28 m³ (-26.23%)
    2. VALVULA_3: -4046.14 m³ (-14.71%)
    3. VALVULA_2: -470.92 m³ (-21.49%)
    4. VALVULA_5: -437.69 m³ (-9.72%)
    5. VALVULA_1: -176.15 m³ (-48.03%)

4. Generando dashboard HTML consolidado...
  ✓ Dashboard guardado: dashboard/Dashboard_Balances_Virtuales.html

ENTREGABLE 4 COMPLETADO

Archivos generados:
  • dashboard/Dashboard_Balances_Virtuales.html - Dashboard principal
  • dashboard/grafica_*.html - Gráficos por válvula
  • dashboard/Alertas_Puntos.csv - Alertas generadas
  • dashboard/Top_Desbalances.csv - Análisis de desbalances
  • dashboard/Top10_Perdidas_Absolutas.csv -

In [16]:
# ============================================================================
# ANÁLISIS DE CONFIABILIDAD DEL MODELO
# ============================================================================
import pandas as pd
import numpy as np
import os

print("=" * 80)
print("ANÁLISIS DE CONFIABILIDAD DEL MODELO")
print("=" * 80)

# Cargar métricas y pronósticos
if os.path.exists('Metrics.csv'):
    df_metrics = pd.read_csv('Metrics.csv', sep=';', decimal=',', encoding='latin-1')
    for col in df_metrics.select_dtypes(include=['object']).columns:
        if col not in ['VALVULA', 'MODELO']:
            df_metrics[col] = pd.to_numeric(df_metrics[col], errors='coerce')
else:
    df_metrics = pd.DataFrame()
    print("⚠ Metrics.csv no encontrado")

if os.path.exists('Pronosticos.csv'):
    df_pronos = pd.read_csv('Pronosticos.csv', sep=';', decimal=',', encoding='latin-1')
else:
    df_pronos = pd.DataFrame()
    print("⚠ Pronosticos.csv no encontrado")

if os.path.exists('Dataset_Train.csv'):
    df_train = pd.read_csv('Dataset_Train.csv', sep=';', decimal=',', encoding='latin-1')
else:
    df_train = pd.DataFrame()

# ===== 1. ANÁLISIS DE MÉTRICAS =====
print("\n1. ANÁLISIS DE MÉTRICAS DE VALIDACIÓN")
print("-" * 80)

if len(df_metrics) > 0:
    # Resumen por modelo
    print("\nMétricas promedio por modelo:")
    resumen_modelos = df_metrics.groupby('MODELO').agg({
        'MAE': ['mean', 'std', 'min', 'max'],
        'MAPE': ['mean', 'std'],
        'RMSE': ['mean', 'std']
    }).round(2)
    print(resumen_modelos)
    
    # Análisis de confiabilidad por métrica
    print("\n📊 Evaluación de Confiabilidad por Métrica:")
    
    # MAE
    mae_promedio = df_metrics['MAE'].mean()
    mae_std = df_metrics['MAE'].std()
    mae_cv = (mae_std / mae_promedio) * 100 if mae_promedio > 0 else np.nan
    
    print(f"\n  MAE:")
    print(f"    Promedio: {mae_promedio:.2f}")
    print(f"    Desviación estándar: {mae_std:.2f}")
    print(f"    Coeficiente de variación: {mae_cv:.2f}%")
    if mae_cv > 100:
        print(f"    ⚠ ALTA VARIABILIDAD: Los errores varían mucho entre válvulas")
    elif mae_cv > 50:
        print(f"    ⚠ VARIABILIDAD MODERADA")
    else:
        print(f"    ✓ Variabilidad aceptable")
    
    # MAPE
    mape_promedio = df_metrics['MAPE'].mean()
    mape_std = df_metrics['MAPE'].std()
    
    print(f"\n  MAPE (Error porcentual promedio):")
    print(f"    Promedio: {mape_promedio:.2f}%")
    print(f"    Desviación estándar: {mape_std:.2f}%")
    
    # Evaluar MAPE
    if mape_promedio < 10:
        print(f"    ✅ EXCELENTE: Error < 10%")
        confiabilidad_mape = "ALTA"
    elif mape_promedio < 20:
        print(f"    ✅ BUENO: Error < 20%")
        confiabilidad_mape = "MEDIA-ALTA"
    elif mape_promedio < 30:
        print(f"    ⚠ ACEPTABLE: Error < 30%")
        confiabilidad_mape = "MEDIA"
    else:
        print(f"    ⚠ BAJO: Error > 30%")
        confiabilidad_mape = "BAJA"
    
    # Análisis por válvula
    print("\n📋 Confiabilidad por Válvula:")
    for v in sorted(df_metrics['VALVULA'].unique()):
        df_v = df_metrics[df_metrics['VALVULA'] == v]
        mejor = df_v.loc[df_v['MAE'].idxmin()]
        peor = df_v.loc[df_v['MAE'].idxmax()]
        
        ratio = peor['MAE'] / mejor['MAE'] if mejor['MAE'] > 0 else np.nan
        
        print(f"\n  {v}:")
        print(f"    Mejor modelo: {mejor['MODELO']} (MAE: {mejor['MAE']:.2f}, MAPE: {mejor['MAPE']:.2f}%)")
        print(f"    Peor modelo: {peor['MODELO']} (MAE: {peor['MAE']:.2f}, MAPE: {peor['MAPE']:.2f}%)")
        print(f"    Ratio mejor/peor: {ratio:.2f}x")
        
        if ratio > 3:
            print(f"    ⚠ ALTA DISPERSIÓN: Los modelos difieren mucho")
        elif ratio > 2:
            print(f"    ⚠ DISPERSIÓN MODERADA")
        else:
            print(f"    ✓ Consenso entre modelos")
        
        # Evaluar MAPE del mejor modelo
        if mejor['MAPE'] < 15:
            print(f"    ✅ Confiabilidad ALTA (MAPE < 15%)")
        elif mejor['MAPE'] < 25:
            print(f"    ⚠ Confiabilidad MEDIA (MAPE 15-25%)")
        else:
            print(f"    ⚠ Confiabilidad BAJA (MAPE > 25%)")

# ===== 2. ANÁLISIS DE DATOS DE ENTRENAMIENTO =====
print("\n2. ANÁLISIS DE DATOS DE ENTRENAMIENTO")
print("-" * 80)

if len(df_train) > 0:
    datos_por_valvula = df_train.groupby('VALVULA').agg({
        'VOLUMEN_ENTRADA_FINAL': 'count',
        'FECHA': ['min', 'max']
    })
    datos_por_valvula.columns = ['NUM_PUNTOS', 'FECHA_MIN', 'FECHA_MAX']
    
    print("\nPuntos históricos por válvula:")
    for v in datos_por_valvula.index:
        n_puntos = datos_por_valvula.loc[v, 'NUM_PUNTOS']
        fecha_min = datos_por_valvula.loc[v, 'FECHA_MIN']
        fecha_max = datos_por_valvula.loc[v, 'FECHA_MAX']
        
        print(f"  {v}: {n_puntos} puntos ({fecha_min} a {fecha_max})")
        
        if n_puntos < 6:
            print(f"    ⚠ POCOS DATOS: < 6 puntos (mínimo recomendado)")
        elif n_puntos < 12:
            print(f"    ⚠ DATOS LIMITADOS: 6-11 puntos (aceptable pero limitado)")
        else:
            print(f"    ✅ DATOS SUFICIENTES: ≥ 12 puntos")
    
    # Análisis de variabilidad en datos históricos
    print("\nVariabilidad en datos históricos:")
    for v in df_train['VALVULA'].unique():
        df_v = df_train[df_train['VALVULA'] == v].sort_values('FECHA')
        valores = df_v['VOLUMEN_ENTRADA_FINAL'].dropna()
        
        if len(valores) > 1:
            cv = (valores.std() / valores.mean()) * 100 if valores.mean() > 0 else np.nan
            print(f"  {v}: CV = {cv:.2f}%")
            
            if cv > 50:
                print(f"    ⚠ ALTA VARIABILIDAD: Datos muy volátiles")
            elif cv > 30:
                print(f"    ⚠ VARIABILIDAD MODERADA")
            else:
                print(f"    ✓ Variabilidad baja (datos estables)")

# ===== 3. ANÁLISIS DEL ENSEMBLE =====
print("\n3. ANÁLISIS DEL ENSEMBLE")
print("-" * 80)

if len(df_pronos) > 0:
    # Verificar si hay problemas con los pesos del ensemble
    print("\n⚠ PROBLEMA DETECTADO: Análisis de pesos del ensemble")
    print("   (Los pesos deberían favorecer al modelo con mejor MAE)")
    
    if len(df_metrics) > 0:
        for v in sorted(df_metrics['VALVULA'].unique()):
            df_v_metrics = df_metrics[df_metrics['VALVULA'] == v]
            mejor_modelo = df_v_metrics.loc[df_v_metrics['MAE'].idxmin(), 'MODELO']
            mejor_mae = df_v_metrics.loc[df_v_metrics['MAE'].idxmin(), 'MAE']
            
            print(f"\n  {v}:")
            print(f"    Mejor modelo (menor MAE): {mejor_modelo} (MAE: {mejor_mae:.2f})")
            
            # Verificar si Prophet está dominando sin ser el mejor
            prophet_mae = df_v_metrics[df_v_metrics['MODELO'] == 'Prophet']['MAE'].values
            if len(prophet_mae) > 0:
                prophet_mae = prophet_mae[0]
                if mejor_modelo != 'Prophet' and prophet_mae > mejor_mae * 1.1:
                    print(f"    ⚠ PROBLEMA: Prophet tiene MAE {prophet_mae:.2f} pero no es el mejor")
                    print(f"       El ensemble debería dar más peso a {mejor_modelo}")

# ===== 4. EVALUACIÓN GENERAL DE CONFIABILIDAD =====
print("\n4. EVALUACIÓN GENERAL DE CONFIABILIDAD")
print("-" * 80)

confiabilidad_puntos = []

if len(df_metrics) > 0:
    for v in sorted(df_metrics['VALVULA'].unique()):
        df_v = df_metrics[df_metrics['VALVULA'] == v]
        mejor = df_v.loc[df_v['MAE'].idxmin()]
        
        # Calcular score de confiabilidad (0-100)
        score = 100
        
        # Penalizar por MAPE alto
        if mejor['MAPE'] > 30:
            score -= 30
        elif mejor['MAPE'] > 20:
            score -= 20
        elif mejor['MAPE'] > 15:
            score -= 10
        
        # Penalizar por alta variabilidad entre modelos
        ratio = df_v['MAE'].max() / df_v['MAE'].min()
        if ratio > 3:
            score -= 20
        elif ratio > 2:
            score -= 10
        
        # Penalizar por pocos datos
        if len(df_train) > 0:
            n_puntos = len(df_train[df_train['VALVULA'] == v])
            if n_puntos < 6:
                score -= 20
            elif n_puntos < 12:
                score -= 10
        
        score = max(0, score)
        
        # Clasificar
        if score >= 80:
            nivel = "ALTA"
            emoji = "✅"
        elif score >= 60:
            nivel = "MEDIA-ALTA"
            emoji = "⚠️"
        elif score >= 40:
            nivel = "MEDIA"
            emoji = "⚠️"
        else:
            nivel = "BAJA"
            emoji = "❌"
        
        confiabilidad_puntos.append({
            'VALVULA': v,
            'SCORE': score,
            'NIVEL': nivel,
            'MEJOR_MODELO': mejor['MODELO'],
            'MAE': mejor['MAE'],
            'MAPE': mejor['MAPE']
        })
        
        print(f"\n{emoji} {v}:")
        print(f"   Score de confiabilidad: {score:.0f}/100 ({nivel})")
        print(f"   Mejor modelo: {mejor['MODELO']} (MAE: {mejor['MAE']:.2f}, MAPE: {mejor['MAPE']:.2f}%)")

# ===== 5. RECOMENDACIONES =====
print("\n5. RECOMENDACIONES")
print("-" * 80)

if len(confiabilidad_puntos) > 0:
    df_conf = pd.DataFrame(confiabilidad_puntos)
    
    # Válvulas con baja confiabilidad
    bajas = df_conf[df_conf['SCORE'] < 60]
    if len(bajas) > 0:
        print("\n⚠ Válvulas con confiabilidad BAJA o MEDIA:")
        for _, row in bajas.iterrows():
            print(f"  • {row['VALVULA']}: Score {row['SCORE']:.0f}/100")
            print(f"    - Revisar calidad de datos históricos")
            print(f"    - Considerar recopilar más datos")
            print(f"    - Validar manualmente las predicciones")
    
    # Promedio general
    score_promedio = df_conf['SCORE'].mean()
    print(f"\n📊 Score promedio de confiabilidad: {score_promedio:.0f}/100")
    
    if score_promedio >= 80:
        print("   ✅ El modelo es GENERALMENTE CONFIABLE")
    elif score_promedio >= 60:
        print("   ⚠️ El modelo es MODERADAMENTE CONFIABLE")
        print("   - Usar con precaución")
        print("   - Validar resultados críticos manualmente")
    else:
        print("   ❌ El modelo tiene CONFIABILIDAD BAJA")
        print("   - NO RECOMENDADO para uso en producción sin mejoras")
        print("   - Revisar datos y modelo")

# Guardar análisis
if len(confiabilidad_puntos) > 0:
    df_conf = pd.DataFrame(confiabilidad_puntos)
    df_conf.to_csv('Analisis_Confiabilidad.csv', index=False, sep=';', decimal=',', encoding='latin-1')
    print(f"\n✓ Análisis guardado en: Analisis_Confiabilidad.csv")

print("\n" + "=" * 80)
print("ANÁLISIS DE CONFIABILIDAD COMPLETADO")
print("=" * 80)



ANÁLISIS DE CONFIABILIDAD DEL MODELO

1. ANÁLISIS DE MÉTRICAS DE VALIDACIÓN
--------------------------------------------------------------------------------

Métricas promedio por modelo:
                  MAE                             MAPE            RMSE  \
                 mean      std     min      max   mean    std     mean   
MODELO                                                                   
CatBoost      1567.08  1646.13   71.41  3602.33  14.18   6.27  1776.49   
LightGBM      2659.22  2873.93  106.21  5758.28  19.02   3.43  2767.18   
RandomForest  1597.86  2132.95  122.28  4739.57  17.45  10.35  1824.53   

                       
                  std  
MODELO                 
CatBoost      1954.20  
LightGBM      3018.05  
RandomForest  2520.32  

📊 Evaluación de Confiabilidad por Métrica:

  MAE:
    Promedio: 1941.39
    Desviación estándar: 2124.53
    Coeficiente de variación: 109.43%
    ⚠ ALTA VARIABILIDAD: Los errores varían mucho entre válvulas

  MAPE (Erro

In [17]:
# Combinar pronósticos con dataset maestro para entregar balance virtual completo
import pandas as pd
import numpy as np

try:
    df_maestro = pd.read_csv('Dataset_Maestro_Balances.csv', sep=';', decimal=',', encoding='latin-1')
    df_fc = pd.read_csv('Pronosticos.csv', sep=';', decimal=',', encoding='latin-1')
except Exception as e:
    print(f"Error cargando archivos: {e}")
    raise

# Normalizar tipos
for col in ['VOLUMEN_ENTRADA_FINAL','VOLUMEN_SALIDA_FINAL','PERDIDAS_FINAL','INDICE_PERDIDAS_FINAL','PRED_ENTRADA','PRED_SALIDA','PRED_PERDIDAS','PRED_INDICE_PERDIDAS']:
    if col in df_maestro.columns:
        df_maestro[col] = pd.to_numeric(df_maestro[col], errors='coerce')
    if col in df_fc.columns:
        df_fc[col] = pd.to_numeric(df_fc[col], errors='coerce')

df_maestro['FECHA'] = pd.to_datetime(df_maestro['FECHA'], errors='coerce')
df_fc['FECHA'] = pd.to_datetime(df_fc['FECHA'], errors='coerce')

# Unir por VALVULA + PERIODO
df_out = df_maestro.merge(df_fc[['VALVULA','PERIODO','PRED_ENTRADA','PRED_SALIDA','PRED_PERDIDAS','PRED_INDICE_PERDIDAS']], on=['VALVULA','PERIODO'], how='left')

# Reemplazar entrada/salida en periodos a predecir
mask_pred = df_out['PERIODO_A_PREDECIR'] == True
df_out.loc[mask_pred, 'VOLUMEN_ENTRADA_FINAL'] = df_out.loc[mask_pred, 'PRED_ENTRADA']
df_out.loc[mask_pred, 'VOLUMEN_SALIDA_FINAL'] = df_out.loc[mask_pred, 'PRED_SALIDA'].fillna(df_out.loc[mask_pred, 'VOLUMEN_SALIDA_FINAL'])

# Recalcular pérdidas e índice para periodos a predecir
df_out.loc[mask_pred, 'PERDIDAS_FINAL'] = df_out.loc[mask_pred, 'VOLUMEN_ENTRADA_FINAL'] - df_out.loc[mask_pred, 'VOLUMEN_SALIDA_FINAL']
df_out.loc[mask_pred, 'INDICE_PERDIDAS_FINAL'] = np.where(df_out.loc[mask_pred, 'VOLUMEN_ENTRADA_FINAL']>0, (df_out.loc[mask_pred, 'PERDIDAS_FINAL']/df_out.loc[mask_pred, 'VOLUMEN_ENTRADA_FINAL'])*100, np.nan)

# Guardar resultado
df_out.to_csv('Predicciones_Con_Balance.csv', index=False, sep=';', decimal=',', encoding='latin-1')
print(f"✓ Balance virtual con predicciones: Predicciones_Con_Balance.csv ({df_out.shape})")
print(df_out[['VALVULA','PERIODO','VOLUMEN_ENTRADA_FINAL','VOLUMEN_SALIDA_FINAL','PERDIDAS_FINAL','INDICE_PERDIDAS_FINAL']].tail(10))

✓ Balance virtual con predicciones: Predicciones_Con_Balance.csv ((82, 26))
      VALVULA  PERIODO  VOLUMEN_ENTRADA_FINAL  VOLUMEN_SALIDA_FINAL  \
72  VALVULA_5   202502            4503.273333              4935.425   
73  VALVULA_5   202503            4503.273333              4866.239   
74  VALVULA_5   202504            4503.273333              4944.441   
75  VALVULA_5   202505            4503.273333              4845.262   
76  VALVULA_5   202506            4503.273333              5483.756   
77  VALVULA_5   202507            4503.273333              4925.483   
78  VALVULA_5   202508            4503.273333              5105.857   
79  VALVULA_5   202509            4503.273333              4679.096   
80  VALVULA_5   202510            4503.273333              4994.496   
81  VALVULA_5   202511            4503.273333              5177.004   

    PERDIDAS_FINAL  INDICE_PERDIDAS_FINAL  
72     -432.151667              -9.596390  
73     -362.965667              -8.060041  
74     -44

## Verificación de Entrenamiento y Predicción

Validar que existan periodos a predecir (`PERIODO_A_PREDECIR=True`) y suficiente historia por válvula para entrenar.

In [18]:
import pandas as pd
import numpy as np

print("="*80)
print("VERIFICACIÓN DE DATOS PARA ENTRENAMIENTO Y PRONÓSTICO")
print("="*80)

df_train = pd.read_csv('Dataset_Train.csv', sep=';', encoding='latin-1')
df_pred = pd.read_csv('Dataset_Prediccion.csv', sep=';', encoding='latin-1')
df_datos_entrada = pd.read_csv('Datos_Entrada.csv', sep=';', encoding='latin-1')

# Conteo de historia por válvula
hist_counts = df_train[df_train['VOLUMEN_ENTRADA_FINAL'].notna()].groupby('VALVULA').size().sort_values(ascending=False)
print("\n1) Historia disponible por válvula (con VOLUMEN_ENTRADA_FINAL):")
print(hist_counts.head(20))
print(f"Total válvulas con historia: {hist_counts.shape[0]}")

# Periodos a predecir por válvula
pred_counts = df_pred.groupby('VALVULA').size().sort_values(ascending=False)
print("\n2) Periodos a predecir por válvula:")
print(pred_counts.head(20))
print(f"Total válvulas con periodos a predecir: {pred_counts.shape[0]}")

# Revisar PERIODO_RETIRO vs máximos periodos
df_datos_entrada['VALVULA'] = df_datos_entrada['CODIGO VALVULA REFERENCIA']
df_datos_entrada['FECHA_RETIRO'] = pd.to_datetime(df_datos_entrada['FECHA RETIRO/TRASLADO'], errors='coerce')
df_datos_entrada['PERIODO_RETIRO'] = df_datos_entrada['FECHA_RETIRO'].dt.to_period('M').astype(str).str.replace('-', '')

# Máximo periodo por válvula en usuarios
df_usuarios = pd.read_csv('Usuarios_Por_Valvula_Simple.csv', sep=';', encoding='latin-1')
df_usuarios['PERIODO'] = df_usuarios['PERIODO'].astype(str)
max_periodos = df_usuarios.groupby('VALVULA')['PERIODO'].max().reset_index().rename(columns={'PERIODO':'MAX_PERIODO_USUARIOS'})

check = df_datos_entrada[['VALVULA','PERIODO_RETIRO']].merge(max_periodos, on='VALVULA', how='left')
check['TIENE_PRED'] = (check['MAX_PERIODO_USUARIOS'] > check['PERIODO_RETIRO'])
print("\n3) Validación retiro vs último periodo de usuarios (debe ser True para tener predicción):")
print(check.head(20))
print(f"Válvulas con posible predicción: {check['TIENE_PRED'].sum()} / {len(check)}")

# Identificar válvulas sin predicción y con poca historia
sin_pred = check[check['TIENE_PRED'] == False]['VALVULA'].tolist()
poca_hist = hist_counts[hist_counts < 6].index.tolist()
print("\n4) Válvulas SIN periodos a predecir:")
print(sin_pred[:20])
print("\n5) Válvulas con POCA historia (<6 puntos):")
print(poca_hist[:20])

VERIFICACIÓN DE DATOS PARA ENTRENAMIENTO Y PRONÓSTICO

1) Historia disponible por válvula (con VOLUMEN_ENTRADA_FINAL):
VALVULA
VALVULA_2    8
VALVULA_1    7
VALVULA_3    7
VALVULA_4    6
VALVULA_5    4
dtype: int64
Total válvulas con historia: 5

2) Periodos a predecir por válvula:
VALVULA
VALVULA_5    13
VALVULA_4    12
VALVULA_2    11
VALVULA_3     5
VALVULA_1     4
dtype: int64
Total válvulas con periodos a predecir: 5

3) Validación retiro vs último periodo de usuarios (debe ser True para tener predicción):
     VALVULA PERIODO_RETIRO MAX_PERIODO_USUARIOS  TIENE_PRED
0  VALVULA_1         202507               202511        True
1  VALVULA_2         202412               202511        True
2  VALVULA_3         202506               202511        True
3  VALVULA_4         202411               202511        True
4  VALVULA_5         202410               202511        True
Válvulas con posible predicción: 5 / 5

4) Válvulas SIN periodos a predecir:
[]

5) Válvulas con POCA historia (<6 pu

## Resumen por Válvula y Gráficas
En esta sección se genera un resumen por válvula del horizonte sin macromedición y se crean gráficas de `VOLUMEN_ENTRADA_FINAL`, `VOLUMEN_SALIDA_FINAL` y `INDICE_PERDIDAS_FINAL` por periodo.

In [19]:
# Resumen por válvula del horizonte sin macromedición
import pandas as pd
import numpy as np

# Cargar balance con predicciones
df = pd.read_csv('Predicciones_Con_Balance.csv', sep=';', decimal=',', encoding='latin-1')
df['FECHA'] = pd.to_datetime(df['FECHA'], errors='coerce')
for col in ['VOLUMEN_ENTRADA_FINAL','VOLUMEN_SALIDA_FINAL','PERDIDAS_FINAL','INDICE_PERDIDAS_FINAL']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Filtrar solo periodos a predecir
df_pred = df[df['PERIODO_A_PREDECIR'] == True].copy()

if df_pred.empty:
    print('⚠ No hay periodos a predecir en Predicciones_Con_Balance.csv')
else:
    resumen = df_pred.groupby('VALVULA').agg({
        'PERIODO': 'count',
        'VOLUMEN_ENTRADA_FINAL': ['sum','mean'],
        'VOLUMEN_SALIDA_FINAL': ['sum','mean'],
        'PERDIDAS_FINAL': ['sum','mean'],
        'INDICE_PERDIDAS_FINAL': 'mean'
    }).reset_index()
    resumen.columns = ['_'.join(col).strip('_') for col in resumen.columns.values]
    resumen.rename(columns={'PERIODO_count':'NUM_PERIODOS'}, inplace=True)
    resumen.to_csv('Resumen_Pronostico_Valvulas.csv', index=False, sep=';', decimal=',', encoding='latin-1')
    print(f"✓ Resumen por válvula guardado: Resumen_Pronostico_Valvulas.csv ({resumen.shape})")
    print(resumen.head(10))

✓ Resumen por válvula guardado: Resumen_Pronostico_Valvulas.csv ((5, 9))
     VALVULA  NUM_PERIODOS  VOLUMEN_ENTRADA_FINAL_sum  \
0  VALVULA_1             4                1466.012352   
1  VALVULA_2            11               22973.972525   
2  VALVULA_3             5              137155.807632   
3  VALVULA_4            12              304519.087600   
4  VALVULA_5            13               58542.553333   

   VOLUMEN_ENTRADA_FINAL_mean  VOLUMEN_SALIDA_FINAL_sum  \
0                  366.503088                 2170.6290   
1                 2088.542957                28154.1200   
2                27431.161526               157386.4889   
3                25376.590633               384838.4250   
4                 4503.273333                64232.5380   

   VOLUMEN_SALIDA_FINAL_mean  PERDIDAS_FINAL_sum  PERDIDAS_FINAL_mean  \
0                 542.657250         -704.616648          -176.154162   
1                2559.465455        -5180.147475          -470.922498   
2         

In [20]:
# Gráficas por válvula del horizonte sin macromedición
import pandas as pd
import plotly.express as px
import os

df = pd.read_csv('Predicciones_Con_Balance.csv', sep=';', decimal=',', encoding='latin-1')
df['FECHA'] = pd.to_datetime(df['FECHA'], errors='coerce')
for col in ['VOLUMEN_ENTRADA_FINAL','VOLUMEN_SALIDA_FINAL','PERDIDAS_FINAL','INDICE_PERDIDAS_FINAL']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Filtrar solo periodos a predecir
df_pred = df[df['PERIODO_A_PREDECIR'] == True].copy()
if df_pred.empty:
    print('⚠ No hay periodos a predecir en Predicciones_Con_Balance.csv')
else:
    os.makedirs('graficas_valvulas', exist_ok=True)
    valvulas = sorted(df_pred['VALVULA'].dropna().unique())
    print(f"Generando gráficas para {len(valvulas)} válvulas...")
    for v in valvulas:
        dv = df_pred[df_pred['VALVULA']==v].sort_values('FECHA')
        if dv.empty:
            continue
        fig1 = px.line(dv, x='FECHA', y=['VOLUMEN_ENTRADA_FINAL','VOLUMEN_SALIDA_FINAL'],
                        title=f'Entrada vs Salida - {v}', labels={'value':'Volumen','variable':'Serie'})
        fig1.write_html(os.path.join('graficas_valvulas', f'{v}_entrada_salida.html'))
        fig1.write_image(os.path.join('graficas_valvulas', f'{v}_entrada_salida.png'))
        fig2 = px.line(dv, x='FECHA', y='INDICE_PERDIDAS_FINAL', title=f'Índice de Pérdidas (%) - {v}', labels={'INDICE_PERDIDAS_FINAL':'% Pérdidas'})
        fig2.write_html(os.path.join('graficas_valvulas', f'{v}_indice_perdidas.html'))
        fig2.write_image(os.path.join('graficas_valvulas', f'{v}_indice_perdidas.png'))
    print("✓ Gráficas guardadas en la carpeta graficas_valvulas/")

Generando gráficas para 5 válvulas...


✓ Gráficas guardadas en la carpeta graficas_valvulas/


## Tabla Comparativa por Válvula
Esta celda genera una tabla comparativa por válvula con KPIs del horizonte sin macromedición y, si están disponibles, métricas de validación de LightGBM.

In [21]:
# Generar tabla comparativa por válvula
import pandas as pd
import numpy as np
import os

# Si no existe Resumen_Pronostico_Valvulas.csv, intentar generarlo desde Predicciones_Con_Balance.csv
if not os.path.exists('Resumen_Pronostico_Valvulas.csv'):
    try:
        df_bal = pd.read_csv('Predicciones_Con_Balance.csv', sep=';', decimal=',', encoding='latin-1')
        for col in ['VOLUMEN_ENTRADA_FINAL','VOLUMEN_SALIDA_FINAL','PERDIDAS_FINAL','INDICE_PERDIDAS_FINAL']:
            if col in df_bal.columns:
                df_bal[col] = pd.to_numeric(df_bal[col], errors='coerce')
        df_pred = df_bal[df_bal.get('PERIODO_A_PREDECIR', False) == True].copy()
        if not df_pred.empty:
            resumen = df_pred.groupby('VALVULA').agg({
                'PERIODO': 'count',
                'VOLUMEN_ENTRADA_FINAL': ['sum','mean'],
                'VOLUMEN_SALIDA_FINAL': ['sum','mean'],
                'PERDIDAS_FINAL': ['sum','mean'],
                'INDICE_PERDIDAS_FINAL': 'mean'
            }).reset_index()
            resumen.columns = ['_'.join(col).strip('_') for col in resumen.columns.values]
            resumen.rename(columns={'PERIODO_count':'NUM_PERIODOS'}, inplace=True)
            resumen.to_csv('Resumen_Pronostico_Valvulas.csv', index=False, sep=';', decimal=',', encoding='latin-1')
            print("✓ Resumen_Pronostico_Valvulas.csv generado desde Predicciones_Con_Balance.csv")
        else:
            print("⚠ No hay periodos a predecir para generar el resumen.")
    except Exception as e:
        print(f"⚠ No se pudo generar el resumen automáticamente: {e}")

# Cargar resumen de pronóstico
df_res = pd.read_csv('Resumen_Pronostico_Valvulas.csv', sep=';', decimal=',', encoding='latin-1')
df_res.rename(columns={
    'VALVULA':'VALVULA',
    'NUM_PERIODOS':'NUM_PERIODOS',
    'VOLUMEN_ENTRADA_FINAL_sum':'ENTRADA_SUM',
    'VOLUMEN_ENTRADA_FINAL_mean':'ENTRADA_MEAN',
    'VOLUMEN_SALIDA_FINAL_sum':'SALIDA_SUM',
    'VOLUMEN_SALIDA_FINAL_mean':'SALIDA_MEAN',
    'PERDIDAS_FINAL_sum':'PERDIDAS_SUM',
    'PERDIDAS_FINAL_mean':'PERDIDAS_MEAN',
    'INDICE_PERDIDAS_FINAL_mean':'INDICE_PERDIDAS_MEAN'
}, inplace=True)

# Intentar cargar métricas de LightGBM si existen
try:
    df_metrics = pd.read_csv('Metrics.csv', sep=';', decimal=',', encoding='latin-1')
    df_metrics = df_metrics[df_metrics['MODELO']=='LightGBM'].copy()
    df_metrics = df_metrics.groupby('VALVULA').agg({'MAE':'mean','RMSE':'mean','MAPE':'mean','MASE':'mean','N_TEST':'sum'}).reset_index()
except Exception:
    df_metrics = None

# Construir comparativo base
cmp = df_res.copy()
cmp['PERDIDAS_%_SOBRE_ENTRADA'] = np.where(cmp['ENTRADA_SUM']>0, (cmp['PERDIDAS_SUM']/cmp['ENTRADA_SUM'])*100, np.nan)
cmp['RELACION_SALIDA_ENTRADA'] = np.where(cmp['ENTRADA_SUM']>0, cmp['SALIDA_SUM']/cmp['ENTRADA_SUM'], np.nan)

# Rankings (mejor = menor pérdidas %)
cmp['RANK_PERDIDAS_%'] = cmp['PERDIDAS_%_SOBRE_ENTRADA'].rank(method='min', ascending=True)
cmp['RANK_INDICE_PERDIDAS_MEAN'] = cmp['INDICE_PERDIDAS_MEAN'].rank(method='min', ascending=True)

# Unir métricas si están disponibles
if df_metrics is not None and not df_metrics.empty:
    cmp = cmp.merge(df_metrics, on='VALVULA', how='left')

# Orden sugerido: por pérdidas % asc
cmp = cmp.sort_values(['PERDIDAS_%_SOBRE_ENTRADA','INDICE_PERDIDAS_MEAN']).reset_index(drop=True)

# Guardar y mostrar
cmp.to_csv('Comparativo_Valvulas.csv', index=False, sep=';', decimal=',', encoding='latin-1')
print(f"✓ Tabla comparativa guardada: Comparativo_Valvulas.csv ({cmp.shape})")
print(cmp.head(10))

✓ Tabla comparativa guardada: Comparativo_Valvulas.csv ((5, 18))
     VALVULA  NUM_PERIODOS    ENTRADA_SUM  ENTRADA_MEAN   SALIDA_SUM  \
0  VALVULA_1             4    1466.012352    366.503088    2170.6290   
1  VALVULA_4            12  304519.087600  25376.590633  384838.4250   
2  VALVULA_2            11   22973.972525   2088.542957   28154.1200   
3  VALVULA_3             5  137155.807632  27431.161526  157386.4889   
4  VALVULA_5            13   58542.553333   4503.273333   64232.5380   

    SALIDA_MEAN  PERDIDAS_SUM  PERDIDAS_MEAN  INDICE_PERDIDAS_MEAN  \
0    542.657250   -704.616648    -176.154162            -48.028511   
1  32069.868750 -80319.337400   -6693.278117            -26.234245   
2   2559.465455  -5180.147475    -470.922498            -21.490073   
3  31477.297780 -20230.681268   -4046.136254            -14.706511   
4   4940.964462  -5689.984667    -437.691128             -9.719400   

   PERDIDAS_%_SOBRE_ENTRADA  RELACION_SALIDA_ENTRADA  RANK_PERDIDAS_%  \
0       

## Dashboard Consolidado
Genera un dashboard HTML con la tabla comparativa y las gráficas por válvula.

In [22]:
# Generar dashboard HTML consolidado
import pandas as pd
import os

# Cargar comparativo
df_cmp = pd.read_csv('Comparativo_Valvulas.csv', sep=';', decimal=',', encoding='latin-1')
os.makedirs('graficas_valvulas', exist_ok=True)
dashboard_path = os.path.join('graficas_valvulas', 'dashboard.html')

# Construir tabla HTML
tabla_html = df_cmp.to_html(index=False, float_format=lambda x: f"{x:,.3f}")

# Construir secciones de gráficas
grafs = []
for v in df_cmp['VALVULA'].dropna().tolist():
    graf1 = f"<h3>{v} - Entrada vs Salida</h3><iframe src='{v}_entrada_salida.html' width='100%' height='400' frameborder='0'></iframe>"
    graf2 = f"<h3>{v} - Índice de Pérdidas</h3><iframe src='{v}_indice_perdidas.html' width='100%' height='400' frameborder='0'></iframe>"
    grafs.append(graf1)
    grafs.append(graf2)

# HTML completo (evitar expresiones entre llaves con f-string)
style_block = """
    body { font-family: Arial, sans-serif; margin: 20px; }
    table { border-collapse: collapse; width: 100%; }
    th, td { border: 1px solid #ddd; padding: 8px; }
    th { background-color: #f2f2f2; }
    h1, h2 { margin-top: 24px; }
    h3 { margin-top: 18px; }
"""

html = """
<!DOCTYPE html>
<html lang='es'>
<head>
  <meta charset='utf-8'/>
  <title>Dashboard Balance Virtual</title>
  <style>
{style}
  </style>
</head>
<body>
  <h1>Dashboard Balance Virtual</h1>
  <h2>Tabla Comparativa por Válvula</h2>
  {tabla}
  <h2>Gráficas por Válvula</h2>
  {graficas}
</body>
</html>
""".format(style=style_block, tabla=tabla_html, graficas=''.join(grafs))

with open(dashboard_path, 'w', encoding='utf-8') as f:
    f.write(html)
print(f"✓ Dashboard generado: {dashboard_path}")

✓ Dashboard generado: graficas_valvulas\dashboard.html


## Limpieza opcional de derivados
Activa `MODO_LIMPIEZA = True` para borrar archivos generados (pronósticos, métricas, datasets derivados y gráficas), conservando solo los datasets iniciales.